# Automozed EPIC validation process against secuTrial data entries

created by: Yasaman Safarkhanlo on 2024.10.07

last modified: file name


In [1]:
import os
import pandas as pd # type: ignore
import numpy as np # type: ignore
from pathlib import Path
from datetime import datetime
import chardet

## Read files


In [2]:
def read_and_modify_secuTrial_export(df):
    df = df.drop([7])                   # Remove row 8 in Excel
    df = df.iloc[6:]                    # Skip the first 6 rows
    df.columns = df.iloc[0]             # Use row 6 as the header
    df = df[1:].reset_index(drop=True)  # Drop the header row and reset index
    return df

def safe_read_file(file_path, custom_reader=None):
    """
    Safely reads a file (Excel or CSV), with an option for a custom reader function.
    
    Parameters:
        file_path (str or Path)
        custom_reader (function, optional)

    Returns:
        pd.DataFrame
    """
    file_path = Path(file_path)
    file_extension = file_path.suffix.lower()

    try:
        # Read based on file extension
        if file_extension in [".xlsx", ".xls"]:
            df = pd.read_excel(file_path, engine='openpyxl' if file_extension == ".xlsx" else 'xlrd', header=None)
        elif file_extension == ".csv":
            df = pd.read_csv(file_path)
        else:
            raise ValueError(f"Unsupported file type: {file_extension}")
        
        # Apply custom reader if provided
        return custom_reader(df) if custom_reader else df

    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")
    except Exception as e:
        print(f"Error reading file at {file_path}: {e}")
    
    return None

In [3]:
def detect_encoding(file_path):
    """
    Detect the encoding of a file using chardet.
    
    Parameters:
        file_path (str or Path): Path to the file
        
    Returns:
        str: Detected encoding
    """
    # Read a sample of the file to detect encoding
    with open(file_path, 'rb') as f:
        raw_data = f.read(10000)  # Read up to 10000 bytes
    result = chardet.detect(raw_data)
    return result['encoding']

In [13]:
#base_dir = Path("/data/app")
base_dir = Path("/Users/yaskhanloo/Developer/bern-storke-center/mb-neuro/data")

# Dynamically find the latest export folder
latest_sT_export = max((base_dir / "sT-files").glob("export-*"), key=lambda x: x.stat().st_mtime, default=None)
latest_EPIC_export = max((base_dir / "EPIC-files").glob("export-*"), key=lambda x: x.stat().st_mtime, default=None)

if latest_sT_export:
    secuTrial_base_dir = latest_sT_export
    REVASC_base_dir = secuTrial_base_dir / "REVASC"
else:
    raise FileNotFoundError("No valid secuTrial export directory found.")

if latest_EPIC_export:
    epic_base_dir = latest_EPIC_export
else:
    raise FileNotFoundError("No valid EPIC export directory found.")

# Define file paths
file_path_secuTrial = secuTrial_base_dir / 'SSR_cases_of_2024.xlsx'
file_path_REVASC = REVASC_base_dir / 'report_SSR01_20250218-105747.xlsx'

#file_path_EPIC = epic_base_dir / 'encounters.xlsx'
file_path_EPIC = epic_base_dir / 'encounters.csv'

# Read files
df_secuTrial = safe_read_file(file_path_secuTrial, custom_reader=read_and_modify_secuTrial_export)
df_REVASC = safe_read_file(file_path_REVASC, custom_reader=read_and_modify_secuTrial_export)

df_EPIC = safe_read_file(file_path_EPIC)

# Print data frame sizes
if df_secuTrial is not None and df_EPIC is not None and df_REVASC is not None:
    print(f'df_secuTrial size: {df_secuTrial.shape}, df_REVASC: {df_REVASC.shape}, df_EPIC size: {df_EPIC.shape}')
else:
    print("Failed to load one or both dataframes.")

/Users/yaskhanloo/Developer/bern-storke-center/mb-neuro/.venv/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/Users/yaskhanloo/Developer/bern-storke-center/mb-neuro/.venv/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Error reading file at /Users/yaskhanloo/Developer/bern-storke-center/mb-neuro/data/EPIC-files/export-20250404/encounters.csv: 'utf-8' codec can't decode byte 0xf6 in position 596: invalid start byte
Failed to load one or both dataframes.


### Merge all sT years into one file


In [5]:
# TO-DO: read and merge all sT export files together

### Merge all EPIC files into one


In [6]:
def merge_single_file(file_path, merge_column, merged_df, prefix=""):
    """
    Merges a single file into the main DataFrame with optional prefixing of columns.
    Handles tab-delimited CSV files.
    
    Parameters:
        file_path (str or Path): Path to file.
        merge_column (str): Column name to merge on.
        merged_df (pd.DataFrame): The main DataFrame to merge into.
        prefix (str): Optional prefix to add to column names for this file.
        
    Returns:
        pd.DataFrame: Updated merged DataFrame.
    """
    # Determine file extension
    file_extension = file_path.suffix.lower()
    
    # Try to detect encoding for CSV files
    detected_encoding = None
    if file_extension == ".csv":
        detected_encoding = detect_encoding(file_path)
        print(f"Detected encoding for {file_path.name}: {detected_encoding}")
    
    # Read file based on its extension
    try:
        if file_extension == ".xlsx":
            df = pd.read_excel(file_path, engine='openpyxl')
        elif file_extension == ".xls":
            df = pd.read_excel(file_path, engine='xlrd')
        elif file_extension == ".csv":
            # Try with the detected encoding and tab delimiter
            try:
                df = pd.read_csv(file_path, encoding=detected_encoding, sep='\t')
            except:
                # If that fails, try common encodings
                encodings_to_try = ['latin1', 'iso-8859-1', 'cp1252', 'utf-8-sig']
                for encoding in encodings_to_try:
                    try:
                        df = pd.read_csv(file_path, encoding=encoding, sep='\t')
                        print(f"Successfully read {file_path.name} with encoding: {encoding}")
                        break
                    except Exception as e:
                        print(f"Failed with encoding {encoding}: {e}")
                else:
                    # Last resort: try with different delimiters
                    delimiters = [',', ';', '|']
                    for delim in delimiters:
                        try:
                            df = pd.read_csv(file_path, encoding='latin1', sep=delim)
                            print(f"Successfully read {file_path.name} with delimiter: '{delim}'")
                            break
                        except Exception as e:
                            print(f"Failed with delimiter '{delim}': {e}")
                    else:
                        raise ValueError(f"Could not read file with any encoding or delimiter")
        else:
            raise ValueError(f"Unsupported file type: {file_extension}")
    except Exception as e:
        print(f"Error reading file {file_path}: {e}")
        return merged_df  # Return the existing DataFrame without merging
    
    # Check if merge column exists
    if merge_column not in df.columns:
        print(f"Warning: Merge column '{merge_column}' not found in {file_path.name}")
        print(f"Available columns: {df.columns.tolist()}")
        return merged_df
        
    # Print number of columns in the current file
    print(f"File: {file_path.name} | Columns: {len(df.columns)} , Rows: {df.shape[0]}")
    
    # Add prefix to columns except the merge column
    df.rename(columns={col: f"{prefix}{col}" for col in df.columns if col != merge_column}, inplace=True)
    
    # Merge the DataFrame into the main DataFrame
    if merged_df.empty:
        return df
    else:
        # Check if merge column exists in both DataFrames
        if merge_column in merged_df.columns:
            return merged_df.merge(df, on=merge_column, how="outer")
        else:
            print(f"Warning: Merge column '{merge_column}' not found in merged DataFrame")
            print(f"Merged DataFrame columns: {merged_df.columns.tolist()}")
            return merged_df

def merge_excel_files(directory, merge_column):
    """
    Merges all EPIC files in a directory based on a specific column, in a defined order.
    
    Parameters:
        directory (str or Path): Directory containing files.
        merge_column (str): Column name to use for merging files.
        
    Returns:
        pd.DataFrame
    """
    # Define merge order and column prefixes
    merge_order = {
        "encounters": "enct.",
        "flowsheet": "flow.",
        "imaging": "img.",
        "lab": "lab.",
        "medication": "med.",
        "monitor": "mon."
    }
    
    directory = Path(directory)
    if not directory.exists():
        raise FileNotFoundError(f"Directory {directory} does not exist.")
    
    merged_df = pd.DataFrame()
    
    # Merge files in the defined order
    for keyword, prefix in merge_order.items():
        files_found = False
        for file in directory.glob(f"*{keyword}*"):
            if file.suffix.lower() in [".xlsx", ".xls", ".csv"]:  # Check for valid file extensions
                files_found = True
                merged_df = merge_single_file(file, merge_column, merged_df, prefix)
        
        if not files_found and keyword != "medication":  # Special handling for medication(s)
            # Try with similar names (e.g., check for "medications" if "medication" not found)
            for file in directory.glob(f"*{keyword}s*"):
                if file.suffix.lower() in [".xlsx", ".xls", ".csv"]:
                    files_found = True
                    merged_df = merge_single_file(file, merge_column, merged_df, prefix)
            
            if not files_found:
                print(f"No files matching '{keyword}' found in {directory}")
    
    # Process any remaining files
    for file in directory.glob("*"):
        if file.suffix.lower() in [".xlsx", ".xls", ".csv"]:
            if not any(keyword in file.stem.lower() for keyword in merge_order):
                merged_df = merge_single_file(file, merge_column, merged_df)
    
    return merged_df

In [7]:
# Directory containing EPIC export files
directory = epic_base_dir

try:
    # First, check what files are actually in the directory
    print("Files found in directory:")
    all_files = list(Path(directory).glob("*"))
    for file in all_files:
        print(f"  - {file.name}")
    
    # Then attempt the merge
    df_EPIC_all = merge_excel_files(directory, merge_column="PAT_ENC_CSN_ID")
    print(f"Merged DataFrame shape: {df_EPIC_all.shape}")
    
    # Show a small sample of the merged data if successful
    if not df_EPIC_all.empty:
        print("\nSample of merged data (first 5 rows):")
        print(df_EPIC_all.head())
        
        # Save the merged dataframe
        output_path = Path(directory) / "merged_epic_data.csv"
        df_EPIC_all.to_csv(output_path, index=False)
        print(f"\nMerged data saved to: {output_path}")
except FileNotFoundError as e:
    print(f"Error: Directory not found - {e}")
except Exception as e:
    print(f"An error occurred: {e}")

Files found in directory:
  - medications.csv
  - lab_results.csv
  - merged_epic_data.csv
  - flowsheet.csv
  - encounters.csv
  - monitoring.csv
  - imaging.csv
Detected encoding for encounters.csv: ISO-8859-1
File: encounters.csv | Columns: 16 , Rows: 2288
Detected encoding for flowsheet.csv: ascii
File: flowsheet.csv | Columns: 31 , Rows: 2288
Detected encoding for imaging.csv: ascii
File: imaging.csv | Columns: 15 , Rows: 2288
Detected encoding for lab_results.csv: ascii
File: lab_results.csv | Columns: 14 , Rows: 2288
Detected encoding for medications.csv: ISO-8859-1
File: medications.csv | Columns: 23 , Rows: 2288
Detected encoding for monitoring.csv: ISO-8859-1
File: monitoring.csv | Columns: 13 , Rows: 2288
Detected encoding for merged_epic_data.csv: utf-8
Available columns: ['PAT_ENC_CSN_ID,enct.name_last,enct.name_first,enct.birth_date,enct.zip,enct.non_swiss,enct.arrival_date,enct.arrival_time,enct.sex,enct.transport,enct.living_pre,enct.height,enct.weight,enct.discharge_de

## Data Cleaning and Merging


### REVASC merge with sT - single year


In [8]:
# Check for unnamed columns in df_secuTrial
unnamed_columns_secuTrial = [col for col in df_secuTrial.columns if not isinstance(col, str) or not col or col.startswith('Unnamed')]
if unnamed_columns_secuTrial:
    print(f'Unnamed columns in df_secuTrial: {unnamed_columns_secuTrial}')
else:
    print('No unnamed columns found in df_secuTrial.')

# Check for unnamed columns in df_REVASC
unnamed_columns_REVASC = [col for col in df_REVASC.columns if not isinstance(col, str) or not col or col.startswith('Unnamed')]
if unnamed_columns_REVASC:
    print(f'Unnamed columns in df_REVASC: {unnamed_columns_REVASC}')
else:
    print('No unnamed columns found in df_REVASC.')

# Merge df_REVASC into df_secuTrial based on Case ID, adding suffix to shared columns
df_secuTrial_w_REVAS = df_secuTrial.merge(
    df_REVASC,
    how='left',
    left_on='Case ID',
    right_on='CaseID',
    suffixes=('', '.revas')  # No suffix for df_secuTrial, '.revas' for df_REVASC
)

df_secuTrial_w_REVAS.drop(columns=['CaseID'], inplace=True, errors='ignore')
df_secuTrial_w_REVAS.reset_index(drop=True, inplace=True)

print(f'df_secuTrial_w_REVAS size: {df_secuTrial_w_REVAS.shape}')

Unnamed columns in df_secuTrial: [nan]
Unnamed columns in df_REVASC: [nan]
df_secuTrial_w_REVAS size: (1795, 429)


### Add FID and SSR


In [9]:
df_EPIC_all['FID'] = df_EPIC_all['img.FID'].fillna(0).astype(int)
df_EPIC_all.insert(0, 'FID', df_EPIC_all.pop('FID'))

df_secuTrial_w_REVAS['SSR'] = df_secuTrial_w_REVAS['Case ID'].str.extract(r'(\d+)$').astype(int)
df_secuTrial_w_REVAS.insert(1, 'SSR', df_secuTrial_w_REVAS.pop('SSR'))
df_secuTrial_w_REVAS = df_secuTrial_w_REVAS.drop(columns=['nan'])

In [10]:
id_log = pd.read_excel(base_dir / 'EPIC2sT-pipeline/Identification_log_SSR_2024_ohne PW.xlsx')

# Set the first row as column names and drop it from the data
id_log.columns = id_log.iloc[0]
id_log = id_log.iloc[1:].reset_index(drop=True)  # Reset index for clarity

# Rename columns for consistency
id_log.rename(columns={'Fall-Nr.(FID)': 'FID', 'SSR Identification SSR-INS-000....': 'SSR'}, inplace=True)

# Merge with df_EPIC_all on 'FID' and reorder columns
df_EPIC_all = df_EPIC_all.merge(id_log[['FID', 'SSR']], on='FID', how='left')
df_EPIC_all.insert(1, 'SSR', df_EPIC_all.pop('SSR'))  # Move 'SSR' to the second column

# Merge with df_secuTrial_w_REVAS on 'SSR' and reorder columns
df_secuTrial_w_REVAS = df_secuTrial_w_REVAS.merge(id_log[['SSR', 'FID']], on='SSR', how='left')
df_secuTrial_w_REVAS.insert(0, 'FID', df_secuTrial_w_REVAS.pop('FID'))  # Move 'FID' to the first column

In [11]:
# Find common column names
common_columns = df_secuTrial_w_REVAS.columns.intersection(df_EPIC_all.columns)

print(common_columns)

Index(['FID', 'SSR'], dtype='object')


In [12]:
# Find common values in 'FID' and 'SSR'
common_values = df_secuTrial_w_REVAS[['FID', 'SSR']].merge(df_EPIC_all[['FID', 'SSR']], on=['FID', 'SSR'], how='inner')

# Filter both DataFrames to keep only matching rows
df_sT_common = df_secuTrial_w_REVAS.merge(common_values, on=['FID', 'SSR'], how='inner')
df_ep_common = df_EPIC_all.merge(common_values, on=['FID', 'SSR'], how='inner')

In [13]:
df_sT_common.shape, df_ep_common.shape

((1447, 430), (1447, 109))

In [14]:
output_dir = base_dir / 'EPIC-export-validation/validation-files'
output_dir.mkdir(parents=True, exist_ok=True)

file_path = output_dir / "merged_lists.xlsx"
with pd.ExcelWriter(file_path) as writer:
    df_sT_common.to_excel(writer, sheet_name="secuTrial_list", index=False)
    df_ep_common.to_excel(writer, sheet_name="EPIC_list", index=False)

print(f"Excel file saved as {file_path}")

Excel file saved as /Users/yaskhanloo/Developer/bern-storke-center/EPIC-export-validation/validation-files/merged_lists.xlsx


In [15]:
# Find rows that exist only in df1
df_sT_only = df_secuTrial_w_REVAS.merge(df_EPIC_all[['FID', 'SSR']], on=['FID', 'SSR'], how='left', indicator=True).query('_merge == "left_only"').drop(columns=['_merge'])

# Find rows that exist only in df2
df_ep_only = df_EPIC_all.merge(df_secuTrial_w_REVAS[['FID', 'SSR']], on=['FID', 'SSR'], how='left', indicator=True).query('_merge == "left_only"').drop(columns=['_merge'])

In [16]:
df_sT_only.shape, df_ep_only.shape

((348, 430), (844, 109))

In [17]:
# Keep only the common columns in df1_only and df2_only
df_sT_only_common_cols = df_sT_only[['FID', 'SSR', 'Last name', 'First name', 'DOB', 'Arrival at hospital']]
df_ep_only_common_cols = df_ep_only[['FID', 'SSR','enct.name_last', 'enct.name_first', 'enct.birth_date', 'enct.arrival_date', 
                                   'mon.INCLUSION_STROKE_DOCUMENTATION', 'mon.INCLUSION_STROKE_FLAG']]

# Save to an Excel file
# Define output directory and ensure it exists
output_dir = base_dir / 'EPIC-export-validation/validation-files'
output_dir.mkdir(parents=True, exist_ok=True)

file_path = output_dir / "missing_patients_2024.xlsx"
with pd.ExcelWriter(file_path) as writer:
    df_sT_only_common_cols.to_excel(writer, sheet_name="secuTrial_list", index=False)
    df_ep_only_common_cols.to_excel(writer, sheet_name="EPIC_list", index=False)

print(f"Excel file saved as {file_path}")

Excel file saved as /Users/yaskhanloo/Developer/bern-storke-center/EPIC-export-validation/validation-files/missing_patients_2024.xlsx


##### find the missing patients from EPIC and sT


In [18]:
# Column mappings
df_sT_column_mapping = {
    "First name": "first_name",
    "Last name": "last_name",
    "DOB": "birth_date",
    "Arrival at hospital": "arrival_date"
}

df_ep_column_mapping = {
    "enct.name_first": "first_name",
    "enct.name_last": "last_name",
    "enct.birth_date": "birth_date",
    "enct.arrival_date": "arrival_date"
}

# Rename columns in both DataFrames to unify them
df1_renamed = df_sT_only.rename(columns=df_sT_column_mapping)
df2_renamed = df_ep_only.rename(columns=df_ep_column_mapping)


In [19]:
df1_renamed['birth_date'] = pd.to_datetime(df1_renamed['birth_date'], errors='coerce').dt.date
df2_renamed['birth_date'] = pd.to_datetime(df2_renamed['birth_date'], errors='coerce').dt.date
df1_renamed['arrival_date'] = pd.to_datetime(df1_renamed['arrival_date'], errors='coerce').dt.date
df2_renamed['arrival_date'] = pd.to_datetime(df2_renamed['arrival_date'], errors='coerce').dt.date

In [20]:
# Ensure columns exist before checking data types
common_columns = ["first_name", "last_name", "birth_date"]

# Function to check and match data types between two DataFrames
def match_column_dtypes(df1, df2, columns):
    for col in columns:
        if col in df1.columns and col in df2.columns:
            if df1[col].dtype != df2[col].dtype:
                # Convert to a common type (string for names, datetime for dates)
                if df1[col].dtype == "object" or df2[col].dtype == "object":
                    df1[col] = df1[col].astype(str)
                    df2[col] = df2[col].astype(str)
                elif "datetime" in str(df1[col].dtype) or "datetime" in str(df2[col].dtype):
                    df1[col] = pd.to_datetime(df1[col], errors='coerce')
                    df2[col] = pd.to_datetime(df2[col], errors='coerce')

# Match data types
match_column_dtypes(df1_renamed, df2_renamed, common_columns)


In [21]:
# Find common rows based on unified columns
df_common_patients = df1_renamed.merge(df2_renamed, on=common_columns, how="inner", suffixes=('_sT', '_ep'))

In [22]:
df_common_patients.shape

(1, 536)

In [23]:
df_common_patients[['SSR_sT', 'FID_sT', 'FID_ep', 'mon.INCLUSION_STROKE_DOCUMENTATION', 'mon.INCLUSION_STROKE_FLAG', 'arrival_date_sT', 'arrival_date_ep'] + common_columns].head(40)

,SSR_sT,FID_sT,FID_ep,mon.INCLUSION_STROKE_DOCUMENTATION,mon.INCLUSION_STROKE_FLAG,arrival_date_sT,arrival_date_ep,first_name,last_name,birth_date
0,14411,10534014,10533127,Yes,Yes,2024-09-09,2024-09-09,Katharina,Schwendimann,1950-04-04


In [24]:
# Define the selected columns to save
selected_columns = ['SSR_sT', 'FID_sT', 'FID_ep', 'mon.INCLUSION_STROKE_DOCUMENTATION', 
                    'mon.INCLUSION_STROKE_FLAG', 'arrival_date_sT', 'arrival_date_ep'] + common_columns

# Ensure df_common_patients exists and contains the required columns
try:
    # Select first 40 rows from the specified columns
    df_to_save = df_common_patients[selected_columns].head(40)

    # Define file path
    file_path = output_dir / "missing_patients_but exist.xlsx"

    # Save to Excel file
    df_to_save.to_excel(file_path, index=False)

    # Provide the download link
    file_path

except NameError:
    print("Error: 'df_common_patients' is not defined. Please reload or redefine it.")
except KeyError as e:
    print(f"Error: Missing columns in 'df_common_patients': {e}")

## Fix data types


In [25]:
# read map file
map_dir = base_dir / 'EPIC2sT-pipeline'
map_file_name = 'map_epic2sT_code_V2_20250224.xlsx'

map_file_path = map_dir / map_file_name

# Load the column mapping Excel file
df_mapping = pd.read_excel(map_file_path)

In [26]:
ep_mapping_dict, ep_type_conversion, ep_category_mappings, ep_datetime_formats = {}, {}, {}, {}
sT_mapping_dict, sT_type_conversion, sT_category_mappings, sT_datetime_formats = {}, {}, {}, {}

In [27]:
def handle_missing_values(x):
    if pd.isna(x) or str(x).strip().lower() in ["", "null", "nan", "<na>", "nat"]:
        return np.nan
    return x

def convert_to_bool(x):
    if pd.isna(x) or str(x).strip().lower() in ["", "null", "nan", "<na>", "nat"]:
        return np.nan
    return str(x).strip().lower() in ["true", "yes", "1"]

def safe_datetime_conversion(s, col_name=None, source=None):
    """
    Converts a column to datetime safely, using specific formats if available.
    """
    # Apply specific formatting if applicable
    if source == "sT" and col_name in sT_datetime_formats:
        date_format = sT_datetime_formats[col_name]
    elif source == "ep" and col_name in ep_datetime_formats:
        date_format = ep_datetime_formats[col_name]
    else:
        date_format = None  # Use default parsing
    
    return pd.to_datetime(s.astype(str).str.strip(), format=date_format, errors="coerce")

In [28]:
def safe_numeric_conversion(series, dtype):
    """Convert series to numeric type safely."""
    series = series.map(handle_missing_values)
    if dtype == "int":
        return pd.to_numeric(series, errors="coerce").astype("Int64")
    elif dtype == "float":
        return pd.to_numeric(series, errors="coerce").astype(float)
    elif dtype == "float-2":
        return pd.to_numeric(series, errors="coerce").round(2)
    return series

##### sT data


In [29]:
def process_secutrial_mapping(row):
    source_file = row["sT_exportFileName"]
    var_name = row["sT_varColumnName"]
    var_dtype = row["sT_varType"]
    var_map = row.get("sT_varMap", None)  

    # Determine column suffix
    suffix = ".revas" if isinstance(source_file, str) and "REVASC" in source_file else ""

    # Ensure var_name is valid before concatenation
    if pd.notna(var_name):
        full_var_name = str(var_name).strip() + suffix
        sT_mapping_dict[full_var_name] = var_name.strip()

        # Define type conversion functions
        if var_dtype == "int":
            sT_type_conversion[full_var_name] = lambda s: pd.to_numeric(
                s.map(handle_missing_values), errors="coerce"
            ).fillna(np.nan).astype("Int64")

        elif var_dtype == "float":
            sT_type_conversion[full_var_name] = lambda s: pd.to_numeric(
                s.map(handle_missing_values), errors="coerce"
            ).astype(float)

        elif var_dtype == "float-2":
            sT_type_conversion[full_var_name] = lambda s: pd.to_numeric(
                s.map(handle_missing_values), errors="coerce"
            ).round(2)

        elif var_dtype == "bool":
            sT_type_conversion[full_var_name] = lambda s: s.map(convert_to_bool)

        elif "datetime" in str(var_dtype):
            format_map = {
                "dd.mm.yyyy": "%d.%m.%Y",
                "yyyy-mm-dd": "%Y-%m-%d",
                "yyyymmdd": "%Y%m%d",
                "hh:mm:ss": "%H:%M:%S",
                "hh:mm": "%H:%M"
            }

            datetime_format = format_map.get(var_map, "%Y%m%d")
            sT_datetime_formats[full_var_name] = datetime_format
            sT_type_conversion[full_var_name] = safe_datetime_conversion

        elif var_dtype == "str":
            sT_type_conversion[full_var_name] = lambda s: s.astype(str).fillna("")


In [30]:
df_secuTrial_w_REVAS_2 = df_secuTrial_w_REVAS.rename(columns=sT_mapping_dict)
for col, converter in sT_type_conversion.items():
    if col in df_secuTrial_w_REVAS_2.columns:
        df_secuTrial_w_REVAS_2[col] = converter(df_secuTrial_w_REVAS_2[col])

##### EPIC data


In [31]:
for _, row in df_mapping.iterrows():
    source_file = row["EPIC_exportFileName"]
    var_name = row["EPIC_varColumnName"]
    var_dtype = row["EPIC_varType"]
    secutrial_var = row.get("sT_varColumnName", None)

    # Prefix mapping
    prefix_map = {
        "encounter": "enct.",
        "flowsheet": "flow.",
        "imaging": "img.",
        "lab": "lab.",
        "medication": "med.",
        "monitor": "mon."
    }
    prefix = next((v for k, v in prefix_map.items() if isinstance(source_file, str) and k.lower() in source_file.lower()), "")

    if pd.notna(var_name):
        full_var_name = prefix + str(var_name)
        ep_mapping_dict[full_var_name] = full_var_name

        if var_dtype == "int":
            ep_type_conversion[full_var_name] = lambda s: pd.to_numeric(
                s.map(handle_missing_values), errors="coerce"
            ).fillna(pd.NA).astype("Int64")  # Fixed conversion to support missing values

        elif var_dtype == "float":
            ep_type_conversion[full_var_name] = lambda s: pd.to_numeric(
                s.map(handle_missing_values), errors="coerce"
            ).astype(float)

        elif var_dtype == "bool":
            ep_type_conversion[full_var_name] = lambda s: s.map(convert_to_bool)

        elif "datetime" in str(var_dtype):
            datetime_format = sT_datetime_formats.get(secutrial_var, "%Y%m%d")
            ep_datetime_formats[full_var_name] = datetime_format
            ep_type_conversion[full_var_name] = lambda s: safe_datetime_conversion(s, full_var_name, "ep")  # ✅ Now properly applied

        elif var_dtype == "str":
            ep_type_conversion[full_var_name] = lambda s: s.astype(str).fillna("")



In [32]:
df_EPIC_all_2 = df_EPIC_all.rename(columns=ep_mapping_dict)

for col, dtype in ep_type_conversion.items():
    if col in df_EPIC_all_2.columns:
        if dtype == "bool":
            df_EPIC_all_2[col] = df_EPIC_all_2[col].map(convert_to_bool)
        elif dtype == "str":
            df_EPIC_all_2[col] = df_EPIC_all_2[col].astype("string").fillna("")
        elif "datetime" in str(dtype):
            df_EPIC_all_2[col] = safe_datetime_conversion(df_EPIC_all_2[col], col, "ep")
        else:
            df_EPIC_all_2[col] = safe_numeric_conversion(df_EPIC_all_2[col], dtype)


## Comparison phase

#### Map Key!!


In [33]:
# Define reusable mappings
yes_no_mapping = {0: 'no', 1: 'yes', False: 'no', True: 'yes'}
bilateral_mapping = {0: 'no', 1: '', 2: 'right', 3: 'left', 4: 'bilateral'}
prosthetic_valves_mapping = {0: 'None', 1: 'Biological', 2: 'Mechanical'}
image_type_mapping = {1: 'CT', 2: 'MRI', 3: 'CT (external)', 4: 'MRI (external)'}
transport_map = {1: 'Ambulance', 2: 'Helicopter', 3: 'Other (taxi,self,relatives,friends...)'}
discharge_dest_map = {
    1: 'Home', 
    3: 'Rehabilitation Hospital', 
    2: 'Other acute care hospital', 
    4: 'Nursing home, palliative care center, or other medical facility'
}

# Define common mappings for multiple columns
yes_no_columns = [
    'flow.iat_stentintracran', 
    'flow.iat_stentextracran', 
    'flow.stroke_pre', 
    'flow.tia_pre', 
    'flow.ich_pre',
    'flow.hypertension', 
    'flow.diabetes', 
    'flow.hyperlipidemia', 
    'flow.smoking', 
    'flow.atrialfib', 
    'flow.chd',
    'flow.lowoutput', 
    'flow.pad', 
    'flow.decompression', 
    'img.iat_mech', 
    'img.follow_mra', 
    'img.follow_cta',
    'img.follow_ultrasound', 
    'img.follow_dsa', 
    'img.follow_tte', 
    'img.follow_tee', 
    'img.follow_holter',
    'med.aspirin_pre', 
    'med.clopidogrel_pre', 
    'med.prasugrel_pre', 
    'med.ticagrelor_pre', 
    'med.dipyridamole_pre',
    'med.vka_pre', 
    'med.rivaroxaban_pre', 
    'med.dabigatran_pre', 
    'med.apixaban_pre', 
    'med.edoxaban_pre',
    'med.parenteralanticg_pre', 
    'med.antihypertensive_pre', 
    'med.antilipid_pre', 
    'med.hormone_pre',
    'med.treat_antiplatelet', 
    'med.treat_anticoagulant', 
    'med.treat_ivt'
]

bilateral_columns = ['flow.mca', 'flow.aca', 'flow.pca', 'flow.vertebrobasilar']

# Define value mappings for specific columns
value_mappings = {
    'enct.non_swiss': {True: 'yes'},
    'enct.sex': {1: 'Male', 2: 'Female'},
    'enct.transport': transport_map,
    'enct.discharge_destinat': discharge_dest_map,  # Double-check mapping values
    'flow.firstangio_result': {2: 'no', 3: 'yes'},  # Double-check mapping values
    'flow.prostheticvalves': prosthetic_valves_mapping,
    'img.firstimage_type': image_type_mapping,
}

# Apply yes_no_mapping and bilateral_mapping to multiple columns dynamically
value_mappings.update({col: yes_no_mapping for col in yes_no_columns})
value_mappings.update({col: bilateral_mapping for col in bilateral_columns})

### compare ep and sT dataset


In [34]:
# main functioning one
def compare_epic_secuTrial(epic_df, secuTrial_df, mapping_df, value_mappings=None):
    """
    Compares values and data types between EPIC and secuTrial DataFrames using a mapping file,
    but only for rows where FID and SSR match in both datasets.

    Args:
        epic_df (DataFrame): The EPIC dataset.
        secuTrial_df (DataFrame): The SecuTrial dataset.
        mapping_df (DataFrame): The mapping Excel file.
        value_mappings (dict, optional): Dictionary for value conversions in EPIC.

    Returns:
        DataFrame: DataFrame with mismatched results in values and data types.
        dict: Dictionary containing percentage statistics.
    """
    if value_mappings is None:
        value_mappings = {}
        
    epic_df.replace(-9999, pd.NA, inplace=True)
    secuTrial_df.replace(-9999, pd.NA, inplace=True)

    # Ensure necessary columns exist before comparison
    if "FID" not in epic_df.columns or "SSR" not in epic_df.columns:
        raise ValueError("EPIC DataFrame must contain 'FID' and 'SSR' columns.")
    if "FID" not in secuTrial_df.columns or "SSR" not in secuTrial_df.columns:
        raise ValueError("SecuTrial DataFrame must contain 'FID' and 'SSR' columns.")

    # Create a set of (FID, SSR) pairs that exist in both DataFrames
    matching_keys = set(epic_df[['FID', 'SSR']].apply(tuple, axis=1)) & set(secuTrial_df[['FID', 'SSR']].apply(tuple, axis=1))

    # Store mismatched results
    mismatched_results = []
    match_count = 0
    secu_missing_count = 0
    epic_missing_count = 0
    mismatch_count = 0
    total_comparisons = 0
    
    # Apply value mappings to EPIC data before comparison and print changes
    for col, mapping in value_mappings.items():
        if col in epic_df.columns:
            #print(f"\n🔹 Applying mapping to column: {col}")
            #print("Before Mapping:")
            #print(epic_df[col].value_counts(dropna=False))  # Show initial value distribution
        
            # Apply mapping
            epic_df[col] = epic_df[col].map(mapping).fillna(epic_df[col])  
            epic_df[col] = epic_df[col].astype(str)  # Ensure string for comparison

            #print("\nAfter Mapping:")
            #print(epic_df[col].value_counts(dropna=False))  # Show transformed value distribution
    
    # Iterate through the mapping file to compare columns
    for _, row in mapping_df.iterrows():
        epic_column_name = row.get('EPIC_varColumnName', None)
        secuTrial_column_name = row.get('sT_varColumnName', None)
        epic_dtype = row.get('EPIC_varType', None)
        secuTrial_dtype = row.get('sT_varType', None)
        column_source = row.get('EPIC_exportFileName', None)
        secu_source = row.get('sT_exportFileName', None)

        if not isinstance(epic_column_name, str) or not isinstance(secuTrial_column_name, str):
            continue  # Skip if column names are missing

        # Determine EPIC column prefix
        prefix = ""
        if isinstance(column_source, str):
            if "encounter" in column_source:
                prefix = "enct."
            elif "flowsheet" in column_source:
                prefix = "flow."
            elif "imaging" in column_source:
                prefix = "img."
            elif "lab" in column_source:
                prefix = "lab."
            elif "medication" in column_source:
                prefix = "med."
            elif "monitor" in column_source:
                prefix = "mon."

        # Determine SecuTrial column suffix
        suffix = ""
        if isinstance(secu_source, str) and "REVASC" in secu_source:
            suffix = ".revas"

        # Construct fully qualified column names
        epic_col = f"{prefix}{epic_column_name}"  # EPIC column with prefix
        secu_col = f"{secuTrial_column_name}{suffix}"  # SecuTrial column with suffix

        # Count missing variables
        if epic_col not in epic_df.columns and secu_col in secuTrial_df.columns:
            epic_missing_count += 1
            continue
        elif secu_col not in secuTrial_df.columns and epic_col in epic_df.columns:
            secu_missing_count += 1
            continue
        elif epic_col not in epic_df.columns and secu_col not in secuTrial_df.columns:
            continue  # Skip comparison if column is missing in both

        total_comparisons += 1

        # Compare values for rows with matching (FID, SSR)
        for fid, ssr in matching_keys:
            epic_value = epic_df.loc[(epic_df["FID"] == fid) & (epic_df["SSR"] == ssr), epic_col].values
            secu_value = secuTrial_df.loc[(secuTrial_df["FID"] == fid) & (secuTrial_df["SSR"] == ssr), secu_col].values

            if len(epic_value) == 0 or len(secu_value) == 0:
                continue  # Skip if value is missing in either DataFrame

            # Convert both values to string before comparing
            epic_value_str = str(epic_value[0])
            secu_value_str = str(secu_value[0])

            # Check if SecuTrial value is NaN or NaT
            if pd.isna(secu_value[0]):
                secu_missing_count += 1
                continue  # Skip further comparison for this case

            if epic_value_str == secu_value_str:
                match_count += 1
            else:  # Store mismatched results
                mismatch_count += 1
                mismatched_results.append({
                    'FID': fid,
                    'SSR': ssr,
                    'EPIC Column': epic_col,
                    'SecuTrial Column': secu_col,
                    'EPIC Value': epic_value_str,
                    'SecuTrial Value': secu_value_str,
                    'EPIC Data Type': str(epic_df[epic_col].dtype),
                    'SecuTrial Data Type': str(secuTrial_df[secu_col].dtype)
                })


    # Calculate percentages
    total_vars = match_count + secu_missing_count + epic_missing_count + mismatch_count
    percentage_stats = {
        "Matching Variables (%)": round((match_count / total_vars) * 100, 2) if total_vars else 0,
        "Variables Missing in EPIC (%)": round((epic_missing_count / total_vars) * 100, 2) if total_vars else 0,
        "Variables Missing in SecuTrial (%)": round((secu_missing_count / total_vars) * 100, 2) if total_vars else 0,
        "Mismatched Variables (%)": round((mismatch_count / total_vars) * 100, 2) if total_vars else 0
    }

    return pd.DataFrame(mismatched_results), percentage_stats

In [35]:
# Claude code - 20250227

def compare_epic_secuTrial(epic_df, secuTrial_df, mapping_df, value_mappings=None):
    """
    Compares values and data types between EPIC and secuTrial DataFrames using a mapping file,
    accounting for data type differences properly.

    Args:
        epic_df (DataFrame): The EPIC dataset.
        secuTrial_df (DataFrame): The SecuTrial dataset.
        mapping_df (DataFrame): The mapping Excel file with data type information.
        value_mappings (dict, optional): Dictionary for value conversions in EPIC.

    Returns:
        DataFrame: DataFrame with mismatched results in values and data types.
        dict: Dictionary containing percentage statistics.
    """
    
    if value_mappings is None:
        value_mappings = {}
        
    # Create working copies to avoid modifying original dataframes
    epic_df_copy = epic_df.copy()
    secuTrial_df_copy = secuTrial_df.copy()
    
    # Replace missing value indicators
    epic_df_copy.replace(-9999, pd.NA, inplace=True)
    secuTrial_df_copy.replace(-9999, pd.NA, inplace=True)

    # Ensure necessary columns exist before comparison
    if "FID" not in epic_df_copy.columns or "SSR" not in epic_df_copy.columns:
        raise ValueError("EPIC DataFrame must contain 'FID' and 'SSR' columns.")
    if "FID" not in secuTrial_df_copy.columns or "SSR" not in secuTrial_df_copy.columns:
        raise ValueError("SecuTrial DataFrame must contain 'FID' and 'SSR' columns.")

    # Create a set of (FID, SSR) pairs that exist in both DataFrames
    matching_keys = set(epic_df_copy[['FID', 'SSR']].apply(tuple, axis=1)) & set(secuTrial_df_copy[['FID', 'SSR']].apply(tuple, axis=1))

    # Store mismatched results
    mismatched_results = []
    match_count = 0
    secu_missing_count = 0
    epic_missing_count = 0
    mismatch_count = 0
    total_comparisons = 0
    
    # Helper function to convert values to the correct type
    def convert_to_type(value, target_type):
        """Convert value to specified type"""
        if pd.isna(value):
            return value
            
        # Handle various data types
        if not isinstance(target_type, str):
            return value  # If no type specified, return as is
            
        if target_type.lower() in ['int', 'integer', 'int64', 'int32']:
            try:
                return int(float(value)) if value != '' else pd.NA
            except (ValueError, TypeError):
                return pd.NA
        elif target_type.lower() in ['float', 'double', 'numeric', 'float64', 'float32']:
            try:
                return float(value) if value != '' else pd.NA
            except (ValueError, TypeError):
                return pd.NA
        elif target_type.lower() in ['date', 'datetime', 'timestamp']:
            try:
                return pd.to_datetime(value) if value else pd.NA
            except (ValueError, TypeError):
                return pd.NA
        elif target_type.lower() in ['bool', 'boolean']:
            if isinstance(value, bool):
                return value
            elif isinstance(value, (int, float)):
                return bool(value)
            elif isinstance(value, str):
                return value.lower() in ['true', 'yes', 'y', '1', 't']
            else:
                return pd.NA
        else:
            # Default to string for text, categorical, etc.
            return str(value) if value is not None else pd.NA
    
    # Helper function to check if values are equivalent
    def equivalent_values(val1, val2, epic_type, secu_type):
        """Compare values with type awareness"""
        # Handle NaN values consistently
        if pd.isna(val1) and pd.isna(val2):
            return True
        elif pd.isna(val1) or pd.isna(val2):
            return False
            
        # Special handling for numeric types
        if epic_type.lower() in ['int', 'integer', 'float', 'double', 'numeric', 'int64', 'int32', 'float64', 'float32'] and \
           secu_type.lower() in ['int', 'integer', 'float', 'double', 'numeric', 'int64', 'int32', 'float64', 'float32']:
            try:
                return abs(float(val1) - float(val2)) < 1e-6
            except (ValueError, TypeError):
                return False
        
        # Date comparison
        elif epic_type.lower() in ['date', 'datetime', 'timestamp'] and \
             secu_type.lower() in ['date', 'datetime', 'timestamp']:
            try:
                dt1 = pd.to_datetime(val1)
                dt2 = pd.to_datetime(val2)
                return dt1 == dt2
            except (ValueError, TypeError):
                return False
                
        # Boolean comparison
        elif epic_type.lower() in ['bool', 'boolean'] and \
             secu_type.lower() in ['bool', 'boolean']:
            bool_truthy = ['true', 'yes', 'y', '1', 't', 'True', 'TRUE', 1, True]
            bool_falsy = ['false', 'no', 'n', '0', 'f', 'False', 'FALSE', 0, False]
            
            val1_bool = val1 in bool_truthy
            val2_bool = val2 in bool_truthy
            return val1_bool == val2_bool
            
        # String comparison (case insensitive)
        elif isinstance(val1, str) and isinstance(val2, str):
            return val1.strip().lower() == val2.strip().lower()
            
        # Default comparison
        else:
            return str(val1) == str(val2)
    
    # Apply value mappings to EPIC data before comparison
    modified_columns = set()
    for col, mapping in value_mappings.items():
        if col in epic_df_copy.columns:
            epic_df_copy[col] = epic_df_copy[col].map(lambda x: mapping.get(x, x))
            modified_columns.add(col)
    
    # Iterate through the mapping file to compare columns
    for _, row in mapping_df.iterrows():
        epic_column_name = row.get('EPIC_varColumnName', None)
        secuTrial_column_name = row.get('sT_varColumnName', None)
        epic_dtype = row.get('EPIC_varType', 'text')  # Default to text if not specified
        secuTrial_dtype = row.get('sT_varType', 'text')  # Default to text if not specified
        column_source = row.get('EPIC_exportFileName', None)
        secu_source = row.get('sT_exportFileName', None)

        if not isinstance(epic_column_name, str) or not isinstance(secuTrial_column_name, str):
            continue  # Skip if column names are missing

        # Determine EPIC column prefix
        prefix = ""
        if isinstance(column_source, str):
            if "encounter" in column_source:
                prefix = "enct."
            elif "flowsheet" in column_source:
                prefix = "flow."
            elif "imaging" in column_source:
                prefix = "img."
            elif "lab" in column_source:
                prefix = "lab."
            elif "medication" in column_source:
                prefix = "med."
            elif "monitor" in column_source:
                prefix = "mon."

        # Determine SecuTrial column suffix
        suffix = ""
        if isinstance(secu_source, str) and "REVASC" in secu_source:
            suffix = ".revas"

        # Construct fully qualified column names
        epic_col = f"{prefix}{epic_column_name}"  # EPIC column with prefix
        secu_col = f"{secuTrial_column_name}{suffix}"  # SecuTrial column with suffix

        # Check if columns exist in respective DataFrames
        if epic_col not in epic_df_copy.columns and secu_col in secuTrial_df_copy.columns:
            epic_missing_count += 1
            continue
        elif secu_col not in secuTrial_df_copy.columns and epic_col in epic_df_copy.columns:
            secu_missing_count += 1
            continue
        elif epic_col not in epic_df_copy.columns and secu_col not in secuTrial_df_copy.columns:
            continue  # Skip comparison if column is missing in both

        total_comparisons += 1

        # Compare values for rows with matching (FID, SSR)
        for fid, ssr in matching_keys:
            epic_row = epic_df_copy.loc[(epic_df_copy["FID"] == fid) & (epic_df_copy["SSR"] == ssr)]
            secu_row = secuTrial_df_copy.loc[(secuTrial_df_copy["FID"] == fid) & (secuTrial_df_copy["SSR"] == ssr)]
            
            if epic_row.empty or secu_row.empty:
                continue  # Skip if no matching row found
                
            epic_value = epic_row[epic_col].iloc[0] if epic_col in epic_row else pd.NA
            secu_value = secu_row[secu_col].iloc[0] if secu_col in secu_row else pd.NA
            
            # Skip comparison if both values are missing
            if pd.isna(epic_value) and pd.isna(secu_value):
                match_count += 1
                continue
                
            # Skip and count as missing if SecuTrial value is NaN
            if pd.isna(secu_value) and not pd.isna(epic_value):
                secu_missing_count += 1
                continue
                
            # Skip and count as missing if EPIC value is NaN
            if pd.isna(epic_value) and not pd.isna(secu_value):
                epic_missing_count += 1
                continue

            # Convert values according to their intended data types
            epic_value_converted = convert_to_type(epic_value, epic_dtype)
            secu_value_converted = convert_to_type(secu_value, secuTrial_dtype)
            
            # Compare values with type awareness
            if equivalent_values(epic_value_converted, secu_value_converted, epic_dtype, secuTrial_dtype):
                match_count += 1
            else:
                mismatch_count += 1
                mismatched_results.append({
                    'FID': fid,
                    'SSR': ssr,
                    'EPIC Column': epic_col,
                    'SecuTrial Column': secu_col,
                    'EPIC Value': str(epic_value),
                    'SecuTrial Value': str(secu_value),
                    'EPIC Expected Type': epic_dtype,
                    'SecuTrial Expected Type': secuTrial_dtype,
                    'EPIC Actual Type': type(epic_value).__name__,
                    'SecuTrial Actual Type': type(secu_value).__name__
                })

    # Calculate percentages
    total_compared = match_count + mismatch_count + secu_missing_count + epic_missing_count
    percentage_stats = {
        "Matching Variables (%)": round((match_count / total_compared) * 100, 2) if total_compared else 0,
        "Variables Missing in EPIC (%)": round((epic_missing_count / total_compared) * 100, 2) if total_compared else 0,
        "Variables Missing in SecuTrial (%)": round((secu_missing_count / total_compared) * 100, 2) if total_compared else 0,
        "Mismatched Variables (%)": round((mismatch_count / total_compared) * 100, 2) if total_compared else 0,
        "Total Comparisons": total_compared,
        "Matches": match_count,
        "EPIC Missing": epic_missing_count,
        "SecuTrial Missing": secu_missing_count,
        "Mismatches": mismatch_count
    }

    return pd.DataFrame(mismatched_results), percentage_stats

In [36]:
# Claude code 2
def compare_epic_secuTrial(epic_df, secuTrial_df, mapping_df, value_mappings=None):
    """
    Compares values and data types between EPIC and secuTrial DataFrames using a mapping file,
    accounting for data type differences properly.

    Args:
        epic_df (DataFrame): The EPIC dataset.
        secuTrial_df (DataFrame): The SecuTrial dataset.
        mapping_df (DataFrame): The mapping Excel file with data type information.
        value_mappings (dict, optional): Dictionary for value conversions in EPIC.

    Returns:
        DataFrame: DataFrame with mismatched results in values and data types.
        dict: Dictionary containing percentage statistics.
    """
    
    if value_mappings is None:
        value_mappings = {}
        
    # Create working copies to avoid modifying original dataframes
    epic_df_copy = epic_df.copy()
    secuTrial_df_copy = secuTrial_df.copy()
    
    # Replace missing value indicators
    epic_df_copy.replace(-9999, pd.NA, inplace=True)
    secuTrial_df_copy.replace(-9999, pd.NA, inplace=True)

    # Ensure necessary columns exist before comparison
    if "FID" not in epic_df_copy.columns or "SSR" not in epic_df_copy.columns:
        raise ValueError("EPIC DataFrame must contain 'FID' and 'SSR' columns.")
    if "FID" not in secuTrial_df_copy.columns or "SSR" not in secuTrial_df_copy.columns:
        raise ValueError("SecuTrial DataFrame must contain 'FID' and 'SSR' columns.")

    # Create a set of (FID, SSR) pairs that exist in both DataFrames
    matching_keys = set(epic_df_copy[['FID', 'SSR']].apply(tuple, axis=1)) & set(secuTrial_df_copy[['FID', 'SSR']].apply(tuple, axis=1))

    # Store mismatched results
    mismatched_results = []
    match_count = 0
    secu_missing_count = 0
    epic_missing_count = 0
    mismatch_count = 0
    total_comparisons = 0
    
    # Helper function to convert values to the correct type
    def convert_to_type(value, target_type):
        """Convert value to specified type"""
        if pd.isna(value):
            return value
            
        # Handle various data types
        if not isinstance(target_type, str):
            return value  # If no type specified, return as is
            
        if target_type.lower() in ['int', 'integer', 'int64', 'int32']:
            try:
                return int(float(value)) if value != '' else pd.NA
            except (ValueError, TypeError):
                return pd.NA
        elif target_type.lower() in ['float', 'double', 'numeric', 'float64', 'float32']:
            try:
                return float(value) if value != '' else pd.NA
            except (ValueError, TypeError):
                return pd.NA
        elif target_type.lower() in ['date', 'datetime', 'timestamp']:
            try:
                return pd.to_datetime(value) if value else pd.NA
            except (ValueError, TypeError):
                return pd.NA
        elif target_type.lower() in ['bool', 'boolean']:
            if isinstance(value, bool):
                return value
            elif isinstance(value, (int, float)):
                return bool(value)
            elif isinstance(value, str):
                return value.lower() in ['true', 'yes', 'y', '1', 't']
            else:
                return pd.NA
        else:
            # Default to string for text, categorical, etc.
            return str(value) if value is not None else pd.NA
    
    # Helper function to check if values are equivalent
    def equivalent_values(val1, val2):
        """Compare values directly"""
        # Handle NaN values consistently
        if pd.isna(val1) and pd.isna(val2):
            return True
        elif pd.isna(val1) or pd.isna(val2):
            return False
            
        # Special handling for numeric types
        if isinstance(val1, (int, float)) and isinstance(val2, (int, float)):
            try:
                return abs(float(val1) - float(val2)) < 1e-6
            except (ValueError, TypeError):
                return False
        
        # Date comparison
        elif isinstance(val1, pd.Timestamp) and isinstance(val2, pd.Timestamp):
            return val1 == val2
                
        # Boolean comparison
        elif isinstance(val1, bool) and isinstance(val2, bool):
            return val1 == val2
            
        # String comparison (case insensitive)
        elif isinstance(val1, str) and isinstance(val2, str):
            return val1.strip().lower() == val2.strip().lower()
            
        # Default comparison
        else:
            return str(val1) == str(val2)
    
    # Build a column mapping dictionary for easier lookups
    column_mappings = {}
    column_types = {}
    
    for _, row in mapping_df.iterrows():
        epic_column_name = row.get('EPIC_varColumnName', None)
        secuTrial_column_name = row.get('sT_varColumnName', None)
        epic_dtype = row.get('EPIC_varType', 'text')  # Default to text if not specified
        secuTrial_dtype = row.get('sT_varType', 'text')  # Default to text if not specified
        column_source = row.get('EPIC_exportFileName', None)
        secu_source = row.get('sT_exportFileName', None)

        if not isinstance(epic_column_name, str) or not isinstance(secuTrial_column_name, str):
            continue  # Skip if column names are missing

        # Determine EPIC column prefix
        prefix = ""
        if isinstance(column_source, str):
            if "encounter" in column_source:
                prefix = "enct."
            elif "flowsheet" in column_source:
                prefix = "flow."
            elif "imaging" in column_source:
                prefix = "img."
            elif "lab" in column_source:
                prefix = "lab."
            elif "medication" in column_source:
                prefix = "med."
            elif "monitor" in column_source:
                prefix = "mon."

        # Determine SecuTrial column suffix
        suffix = ""
        if isinstance(secu_source, str) and "REVASC" in secu_source:
            suffix = ".revas"

        # Construct fully qualified column names
        epic_col = f"{prefix}{epic_column_name}"  # EPIC column with prefix
        secu_col = f"{secuTrial_column_name}{suffix}"  # SecuTrial column with suffix
        
        # Store the mapping
        column_mappings[epic_col] = secu_col
        column_types[epic_col] = {'epic_type': epic_dtype, 'secu_type': secuTrial_dtype}
    
    # First, apply value mappings to EPIC data
    for col, mapping in value_mappings.items():
        if col in epic_df_copy.columns:
            epic_df_copy[col] = epic_df_copy[col].map(lambda x: mapping.get(x, x))
    
    # Next, convert EPIC data types to match secuTrial types
    for epic_col, secu_col in column_mappings.items():
        if epic_col in epic_df_copy.columns and secu_col in secuTrial_df_copy.columns:
            target_type = column_types[epic_col]['secu_type']
            epic_df_copy[epic_col] = epic_df_copy[epic_col].apply(lambda x: convert_to_type(x, target_type))
    
    # Now compare the values
    for epic_col, secu_col in column_mappings.items():
        # Check if columns exist in respective DataFrames
        if epic_col not in epic_df_copy.columns and secu_col in secuTrial_df_copy.columns:
            epic_missing_count += 1
            continue
        elif secu_col not in secuTrial_df_copy.columns and epic_col in epic_df_copy.columns:
            secu_missing_count += 1
            continue
        elif epic_col not in epic_df_copy.columns and secu_col not in secuTrial_df_copy.columns:
            continue  # Skip comparison if column is missing in both

        total_comparisons += 1
        
        # Get data types for this column pair
        epic_dtype = column_types[epic_col]['epic_type']
        secuTrial_dtype = column_types[epic_col]['secu_type']

        # Compare values for rows with matching (FID, SSR)
        for fid, ssr in matching_keys:
            epic_row = epic_df_copy.loc[(epic_df_copy["FID"] == fid) & (epic_df_copy["SSR"] == ssr)]
            secu_row = secuTrial_df_copy.loc[(secuTrial_df_copy["FID"] == fid) & (secuTrial_df_copy["SSR"] == ssr)]
            
            if epic_row.empty or secu_row.empty:
                continue  # Skip if no matching row found
                
            epic_value = epic_row[epic_col].iloc[0] if epic_col in epic_row else pd.NA
            secu_value = secu_row[secu_col].iloc[0] if secu_col in secu_row else pd.NA
            
            # Skip comparison if both values are missing
            if pd.isna(epic_value) and pd.isna(secu_value):
                match_count += 1
                continue
                
            # Skip and count as missing if SecuTrial value is NaN
            if pd.isna(secu_value) and not pd.isna(epic_value):
                secu_missing_count += 1
                continue
                
            # Skip and count as missing if EPIC value is NaN
            if pd.isna(epic_value) and not pd.isna(secu_value):
                epic_missing_count += 1
                continue

            # Compare values directly (no need to convert again)
            if equivalent_values(epic_value, secu_value):
                match_count += 1
            else:
                mismatch_count += 1
                mismatched_results.append({
                    'FID': fid,
                    'SSR': ssr,
                    'EPIC Column': epic_col,
                    'SecuTrial Column': secu_col,
                    'EPIC Value': str(epic_value),
                    'SecuTrial Value': str(secu_value),
                    'EPIC Expected Type': epic_dtype,
                    'SecuTrial Expected Type': secuTrial_dtype,
                    'EPIC Actual Type': type(epic_value).__name__,
                    'SecuTrial Actual Type': type(secu_value).__name__
                })

    # Calculate percentages
    total_compared = match_count + mismatch_count + secu_missing_count + epic_missing_count
    percentage_stats = {
        "Matching Variables (%)": round((match_count / total_compared) * 100, 2) if total_compared else 0,
        "Variables Missing in EPIC (%)": round((epic_missing_count / total_compared) * 100, 2) if total_compared else 0,
        "Variables Missing in SecuTrial (%)": round((secu_missing_count / total_compared) * 100, 2) if total_compared else 0,
        "Mismatched Variables (%)": round((mismatch_count / total_compared) * 100, 2) if total_compared else 0,
        "Total Comparisons": total_compared,
        "Matches": match_count,
        "EPIC Missing": epic_missing_count,
        "SecuTrial Missing": secu_missing_count,
        "Mismatches": mismatch_count
    }

    return pd.DataFrame(mismatched_results), percentage_stats

In [37]:
# Claude code 3
def compare_epic_secuTrial(epic_df, secuTrial_df, mapping_df, value_mappings=None):
    """
    Compares values and data types between EPIC and secuTrial DataFrames using a mapping file,
    accounting for data type differences properly.

    Args:
        epic_df (DataFrame): The EPIC dataset.
        secuTrial_df (DataFrame): The SecuTrial dataset.
        mapping_df (DataFrame): The mapping Excel file with data type information.
        value_mappings (dict, optional): Dictionary for value conversions in EPIC.

    Returns:
        DataFrame: DataFrame with mismatched results in values and data types.
        dict: Dictionary containing percentage statistics.
    """
    
    import pandas as pd
    import re
    
    if value_mappings is None:
        value_mappings = {}
        
    # Create working copies to avoid modifying original dataframes
    epic_df_copy = epic_df.copy()
    secuTrial_df_copy = secuTrial_df.copy()
    
    # Replace missing value indicators
    epic_df_copy.replace(-9999, pd.NA, inplace=True)
    secuTrial_df_copy.replace(-9999, pd.NA, inplace=True)

    # Ensure necessary columns exist before comparison
    if "FID" not in epic_df_copy.columns or "SSR" not in epic_df_copy.columns:
        raise ValueError("EPIC DataFrame must contain 'FID' and 'SSR' columns.")
    if "FID" not in secuTrial_df_copy.columns or "SSR" not in secuTrial_df_copy.columns:
        raise ValueError("SecuTrial DataFrame must contain 'FID' and 'SSR' columns.")

    # Create a set of (FID, SSR) pairs that exist in both DataFrames
    matching_keys = set(epic_df_copy[['FID', 'SSR']].apply(tuple, axis=1)) & set(secuTrial_df_copy[['FID', 'SSR']].apply(tuple, axis=1))

    # Store mismatched results
    mismatched_results = []
    match_count = 0
    secu_missing_count = 0
    epic_missing_count = 0
    mismatch_count = 0
    total_comparisons = 0
    
    # Helper function to convert values to the correct type
    def convert_to_type(value, target_type):
        """Convert value to specified type with specific formatting"""
        if pd.isna(value):
            return value
            
        # Handle various data types
        if not isinstance(target_type, str):
            return value  # If no type specified, return as is
            
        # Check for float with decimal specification (e.g., float-1, float-2)
        float_match = re.match(r'float-(\d+)', target_type.lower())
        if float_match:
            try:
                decimal_places = int(float_match.group(1))
                if value == '':
                    return pd.NA
                float_val = float(value)
                return round(float_val, decimal_places)
            except (ValueError, TypeError):
                return pd.NA
        
        if target_type.lower() in ['int', 'integer', 'int64', 'int32']:
            try:
                return int(float(value)) if value != '' else pd.NA
            except (ValueError, TypeError):
                return pd.NA
        elif target_type.lower() in ['float', 'double', 'numeric', 'float64', 'float32']:
            try:
                return float(value) if value != '' else pd.NA
            except (ValueError, TypeError):
                return pd.NA
        elif target_type.lower() in ['date', 'datetime', 'timestamp']:
            try:
                if value == '':
                    return pd.NA
                # Convert to datetime and then to yyyymmdd hh:mm format
                dt = pd.to_datetime(value)
                return dt.strftime('%Y%m%d %H:%M')
            except (ValueError, TypeError, AttributeError):
                return pd.NA
        elif target_type.lower() in ['bool', 'boolean']:
            if isinstance(value, bool):
                return value
            elif isinstance(value, (int, float)):
                return bool(value)
            elif isinstance(value, str):
                return value.lower() in ['true', 'yes', 'y', '1', 't']
            else:
                return pd.NA
        else:
            # Default to string for text, categorical, etc.
            return str(value) if value is not None else pd.NA
    
    # Helper function to check if values are equivalent
    def equivalent_values(val1, val2, target_type):
        """Compare values with formatted type awareness"""
        # Handle NaN values consistently
        if pd.isna(val1) and pd.isna(val2):
            return True
        elif pd.isna(val1) or pd.isna(val2):
            return False
            
        # Check for float with decimal specification (e.g., float-1, float-2)
        float_match = re.match(r'float-(\d+)', target_type.lower()) if isinstance(target_type, str) else None
        if float_match:
            try:
                decimal_places = int(float_match.group(1))
                val1_rounded = round(float(val1), decimal_places)
                val2_rounded = round(float(val2), decimal_places)
                return val1_rounded == val2_rounded
            except (ValueError, TypeError):
                return False
        
        # Special handling for numeric types
        if isinstance(val1, (int, float)) and isinstance(val2, (int, float)):
            try:
                return abs(float(val1) - float(val2)) < 1e-6
            except (ValueError, TypeError):
                return False
        
        # Date comparison (already in string format)
        if isinstance(target_type, str) and target_type.lower() in ['date', 'datetime', 'timestamp']:
            return val1 == val2
                
        # Boolean comparison
        elif isinstance(val1, bool) and isinstance(val2, bool):
            return val1 == val2
            
        # String comparison (case insensitive)
        elif isinstance(val1, str) and isinstance(val2, str):
            return val1.strip().lower() == val2.strip().lower()
            
        # Default comparison
        else:
            return str(val1) == str(val2)
    
    # Build a column mapping dictionary for easier lookups
    column_mappings = {}
    column_types = {}
    
    for _, row in mapping_df.iterrows():
        epic_column_name = row.get('EPIC_varColumnName', None)
        secuTrial_column_name = row.get('sT_varColumnName', None)
        epic_dtype = row.get('EPIC_varType', 'text')  # Default to text if not specified
        secuTrial_dtype = row.get('sT_varType', 'text')  # Default to text if not specified
        column_source = row.get('EPIC_exportFileName', None)
        secu_source = row.get('sT_exportFileName', None)

        if not isinstance(epic_column_name, str) or not isinstance(secuTrial_column_name, str):
            continue  # Skip if column names are missing

        # Determine EPIC column prefix
        prefix = ""
        if isinstance(column_source, str):
            if "encounter" in column_source:
                prefix = "enct."
            elif "flowsheet" in column_source:
                prefix = "flow."
            elif "imaging" in column_source:
                prefix = "img."
            elif "lab" in column_source:
                prefix = "lab."
            elif "medication" in column_source:
                prefix = "med."
            elif "monitor" in column_source:
                prefix = "mon."

        # Determine SecuTrial column suffix
        suffix = ""
        if isinstance(secu_source, str) and "REVASC" in secu_source:
            suffix = ".revas"

        # Construct fully qualified column names
        epic_col = f"{prefix}{epic_column_name}"  # EPIC column with prefix
        secu_col = f"{secuTrial_column_name}{suffix}"  # SecuTrial column with suffix
        
        # If secuTrial type is int, override EPIC type to also be int
        if secuTrial_dtype.lower() in ['int', 'integer', 'int64', 'int32']:
            epic_dtype = 'int'
        # If secuTrial type is float or float-n, override EPIC type
        elif secuTrial_dtype.lower() in ['float', 'double', 'numeric', 'float64', 'float32'] or re.match(r'float-\d+', secuTrial_dtype.lower()):
            epic_dtype = secuTrial_dtype
        
        # Store the mapping
        column_mappings[epic_col] = secu_col
        column_types[epic_col] = {'epic_type': epic_dtype, 'secu_type': secuTrial_dtype}
    
    # First, apply value mappings to EPIC data
    for col, mapping in value_mappings.items():
        if col in epic_df_copy.columns:
            epic_df_copy[col] = epic_df_copy[col].map(lambda x: mapping.get(x, x))
    
    # Next, convert data types in both dataframes
    for epic_col, secu_col in column_mappings.items():
        # For EPIC dataframe
        if epic_col in epic_df_copy.columns:
            target_type = column_types[epic_col]['epic_type']
            epic_df_copy[epic_col] = epic_df_copy[epic_col].apply(lambda x: convert_to_type(x, target_type))
        
        # For secuTrial dataframe
        if secu_col in secuTrial_df_copy.columns:
            target_type = column_types[epic_col]['secu_type']
            secuTrial_df_copy[secu_col] = secuTrial_df_copy[secu_col].apply(lambda x: convert_to_type(x, target_type))
    
    # Now compare the values
    for epic_col, secu_col in column_mappings.items():
        # Check if columns exist in respective DataFrames
        if epic_col not in epic_df_copy.columns and secu_col in secuTrial_df_copy.columns:
            epic_missing_count += 1
            continue
        elif secu_col not in secuTrial_df_copy.columns and epic_col in epic_df_copy.columns:
            secu_missing_count += 1
            continue
        elif epic_col not in epic_df_copy.columns and secu_col not in secuTrial_df_copy.columns:
            continue  # Skip comparison if column is missing in both

        total_comparisons += 1
        
        # Get target type for this column
        target_type = column_types[epic_col]['secu_type']  # Use secuTrial type as the target

        # Compare values for rows with matching (FID, SSR)
        for fid, ssr in matching_keys:
            epic_row = epic_df_copy.loc[(epic_df_copy["FID"] == fid) & (epic_df_copy["SSR"] == ssr)]
            secu_row = secuTrial_df_copy.loc[(secuTrial_df_copy["FID"] == fid) & (secuTrial_df_copy["SSR"] == ssr)]
            
            if epic_row.empty or secu_row.empty:
                continue  # Skip if no matching row found
                
            epic_value = epic_row[epic_col].iloc[0] if epic_col in epic_row.columns else pd.NA
            secu_value = secu_row[secu_col].iloc[0] if secu_col in secu_row.columns else pd.NA
            
            # Skip comparison if both values are missing
            if pd.isna(epic_value) and pd.isna(secu_value):
                match_count += 1
                continue
                
            # Skip and count as missing if SecuTrial value is NaN
            if pd.isna(secu_value) and not pd.isna(epic_value):
                secu_missing_count += 1
                continue
                
            # Skip and count as missing if EPIC value is NaN
            if pd.isna(epic_value) and not pd.isna(secu_value):
                epic_missing_count += 1
                continue

            # Compare values using the target type
            if equivalent_values(epic_value, secu_value, target_type):
                match_count += 1
            else:
                mismatch_count += 1
                mismatched_results.append({
                    'FID': fid,
                    'SSR': ssr,
                    'EPIC Column': epic_col,
                    'SecuTrial Column': secu_col,
                    'EPIC Value': str(epic_value),
                    'SecuTrial Value': str(secu_value),
                    'EPIC Expected Type': column_types[epic_col]['epic_type'],
                    'SecuTrial Expected Type': target_type,
                    'EPIC Actual Type': type(epic_value).__name__,
                    'SecuTrial Actual Type': type(secu_value).__name__
                })

    # Calculate percentages
    total_compared = match_count + mismatch_count + secu_missing_count + epic_missing_count
    percentage_stats = {
        "Matching Variables (%)": round((match_count / total_compared) * 100, 2) if total_compared else 0,
        "Variables Missing in EPIC (%)": round((epic_missing_count / total_compared) * 100, 2) if total_compared else 0,
        "Variables Missing in SecuTrial (%)": round((secu_missing_count / total_compared) * 100, 2) if total_compared else 0,
        "Mismatched Variables (%)": round((mismatch_count / total_compared) * 100, 2) if total_compared else 0,
        "Total Comparisons": total_compared,
        "Matches": match_count,
        "EPIC Missing": epic_missing_count,
        "SecuTrial Missing": secu_missing_count,
        "Mismatches": mismatch_count
    }

    return pd.DataFrame(mismatched_results), percentage_stats

In [38]:
# Claude code 4
def compare_epic_secuTrial(epic_df, secuTrial_df, mapping_df, value_mappings=None):
    """
    Compares values and data types between EPIC and secuTrial DataFrames using a mapping file,
    accounting for data type differences properly.

    Args:
        epic_df (DataFrame): The EPIC dataset.
        secuTrial_df (DataFrame): The SecuTrial dataset.
        mapping_df (DataFrame): The mapping Excel file with data type information.
        value_mappings (dict, optional): Dictionary for value conversions in EPIC.

    Returns:
        DataFrame: DataFrame with mismatched results in values and data types.
        dict: Dictionary containing percentage statistics.
    """
    
    import pandas as pd
    import re
    
    if value_mappings is None:
        value_mappings = {}
        
    # Create working copies to avoid modifying original dataframes
    epic_df_copy = epic_df.copy()
    secuTrial_df_copy = secuTrial_df.copy()
    
    # Replace missing value indicators
    epic_df_copy.replace(-9999, pd.NA, inplace=True)
    secuTrial_df_copy.replace(-9999, pd.NA, inplace=True)

    # Ensure necessary columns exist before comparison
    if "FID" not in epic_df_copy.columns or "SSR" not in epic_df_copy.columns:
        raise ValueError("EPIC DataFrame must contain 'FID' and 'SSR' columns.")
    if "FID" not in secuTrial_df_copy.columns or "SSR" not in secuTrial_df_copy.columns:
        raise ValueError("SecuTrial DataFrame must contain 'FID' and 'SSR' columns.")

    # Create a set of (FID, SSR) pairs that exist in both DataFrames
    matching_keys = set(epic_df_copy[['FID', 'SSR']].apply(tuple, axis=1)) & set(secuTrial_df_copy[['FID', 'SSR']].apply(tuple, axis=1))

    # Store mismatched results
    mismatched_results = []
    match_count = 0
    secu_missing_count = 0
    epic_missing_count = 0
    mismatch_count = 0
    total_comparisons = 0
    
    # Helper function to standardize boolean values
    def standardize_boolean(value):
        if pd.isna(value):
            return pd.NA
        
        if isinstance(value, bool):
            return "yes" if value else "no"
        elif isinstance(value, (int, float)):
            return "yes" if value else "no"
        elif isinstance(value, str):
            if value.lower() in ['true', 'yes', 'y', '1', 't']:
                return "yes"
            elif value.lower() in ['false', 'no', 'n', '0', 'f']:
                return "no"
        
        return str(value)
    
    # Helper function to convert values to the correct type
    def convert_to_type(value, target_type):
        """Convert value to specified type with specific formatting"""
        if pd.isna(value):
            return pd.NA
            
        # Handle various data types
        if not isinstance(target_type, str):
            return value  # If no type specified, return as is
            
        # Check for float with decimal specification (e.g., float-1, float-2)
        float_match = re.match(r'float-(\d+)', target_type.lower())
        if float_match:
            try:
                decimal_places = int(float_match.group(1))
                if value == '':
                    return pd.NA
                float_val = float(value)
                return round(float_val, decimal_places)
            except (ValueError, TypeError):
                return pd.NA
        
        if target_type.lower() in ['int', 'integer', 'int64', 'int32']:
            try:
                return int(float(value)) if value != '' else pd.NA
            except (ValueError, TypeError):
                return pd.NA
        elif target_type.lower() in ['float', 'double', 'numeric', 'float64', 'float32']:
            try:
                return float(value) if value != '' else pd.NA
            except (ValueError, TypeError):
                return pd.NA
        elif target_type.lower() in ['date', 'datetime', 'timestamp']:
            try:
                if value == '':
                    return pd.NA
                # Convert to datetime and then to yyyymmdd hh:mm format
                dt = pd.to_datetime(value)
                return dt.strftime('%Y%m%d %H:%M')
            except (ValueError, TypeError, AttributeError):
                return pd.NA
        elif target_type.lower() in ['bool', 'boolean']:
            return standardize_boolean(value)
        else:
            # Default to string for text, categorical, etc.
            return str(value) if value is not None and value != '' else pd.NA
    
    # Helper function to check if values are equivalent
    def equivalent_values(val1, val2, target_type):
        """Compare values with formatted type awareness"""
        # Handle NaN values consistently
        if pd.isna(val1) and pd.isna(val2):
            return True
        elif pd.isna(val1) or pd.isna(val2):
            return False
            
        # Check for float with decimal specification (e.g., float-1, float-2)
        float_match = re.match(r'float-(\d+)', target_type.lower()) if isinstance(target_type, str) else None
        if float_match:
            try:
                decimal_places = int(float_match.group(1))
                val1_rounded = round(float(val1), decimal_places)
                val2_rounded = round(float(val2), decimal_places)
                return val1_rounded == val2_rounded
            except (ValueError, TypeError):
                return False
        
        # Boolean comparison (standardized to yes/no)
        if isinstance(target_type, str) and target_type.lower() in ['bool', 'boolean']:
            val1_std = standardize_boolean(val1)
            val2_std = standardize_boolean(val2)
            return val1_std == val2_std
        
        # Special handling for numeric types
        if isinstance(val1, (int, float)) and isinstance(val2, (int, float)):
            try:
                return abs(float(val1) - float(val2)) < 1e-6
            except (ValueError, TypeError):
                return False
        
        # Date comparison (already in string format)
        if isinstance(target_type, str) and target_type.lower() in ['date', 'datetime', 'timestamp']:
            return val1 == val2
            
        # String comparison (case insensitive)
        elif isinstance(val1, str) and isinstance(val2, str):
            return val1.strip().lower() == val2.strip().lower()
            
        # Default comparison
        else:
            return str(val1) == str(val2)
    
    # Build a column mapping dictionary for easier lookups
    column_mappings = {}
    column_types = {}
    
    for _, row in mapping_df.iterrows():
        epic_column_name = row.get('EPIC_varColumnName', None)
        secuTrial_column_name = row.get('sT_varColumnName', None)
        epic_dtype = row.get('EPIC_varType', 'text')  # Default to text if not specified
        secuTrial_dtype = row.get('sT_varType', 'text')  # Default to text if not specified
        column_source = row.get('EPIC_exportFileName', None)
        secu_source = row.get('sT_exportFileName', None)

        if not isinstance(epic_column_name, str) or not isinstance(secuTrial_column_name, str):
            continue  # Skip if column names are missing

        # Determine EPIC column prefix
        prefix = ""
        if isinstance(column_source, str):
            if "encounter" in column_source:
                prefix = "enct."
            elif "flowsheet" in column_source:
                prefix = "flow."
            elif "imaging" in column_source:
                prefix = "img."
            elif "lab" in column_source:
                prefix = "lab."
            elif "medication" in column_source:
                prefix = "med."
            elif "monitor" in column_source:
                prefix = "mon."

        # Determine SecuTrial column suffix
        suffix = ""
        if isinstance(secu_source, str) and "REVASC" in secu_source:
            suffix = ".revas"

        # Construct fully qualified column names
        epic_col = f"{prefix}{epic_column_name}"  # EPIC column with prefix
        secu_col = f"{secuTrial_column_name}{suffix}"  # SecuTrial column with suffix
        
        # If secuTrial type is int, override EPIC type to also be int
        if secuTrial_dtype.lower() in ['int', 'integer', 'int64', 'int32']:
            epic_dtype = 'int'
        # If secuTrial type is float or float-n, override EPIC type
        elif secuTrial_dtype.lower() in ['float', 'double', 'numeric', 'float64', 'float32'] or re.match(r'float-\d+', secuTrial_dtype.lower()):
            epic_dtype = secuTrial_dtype
        
        # Store the mapping
        column_mappings[epic_col] = secu_col
        column_types[epic_col] = {'epic_type': epic_dtype, 'secu_type': secuTrial_dtype}
    
    # First, apply value mappings to EPIC data
    for col, mapping in value_mappings.items():
        if col in epic_df_copy.columns:
            epic_df_copy[col] = epic_df_copy[col].map(lambda x: mapping.get(x, x))
    
    # Next, convert data types in both dataframes
    for epic_col, secu_col in column_mappings.items():
        # For EPIC dataframe
        if epic_col in epic_df_copy.columns:
            target_type = column_types[epic_col]['epic_type']
            epic_df_copy[epic_col] = epic_df_copy[epic_col].apply(lambda x: convert_to_type(x, target_type))
        
        # For secuTrial dataframe
        if secu_col in secuTrial_df_copy.columns:
            target_type = column_types[epic_col]['secu_type']
            secuTrial_df_copy[secu_col] = secuTrial_df_copy[secu_col].apply(lambda x: convert_to_type(x, target_type))
    
    # Now compare the values
    for epic_col, secu_col in column_mappings.items():
        # Check if columns exist in respective DataFrames
        if epic_col not in epic_df_copy.columns and secu_col in secuTrial_df_copy.columns:
            epic_missing_count += 1
            continue
        elif secu_col not in secuTrial_df_copy.columns and epic_col in epic_df_copy.columns:
            secu_missing_count += 1
            continue
        elif epic_col not in epic_df_copy.columns and secu_col not in secuTrial_df_copy.columns:
            continue  # Skip comparison if column is missing in both

        total_comparisons += 1
        
        # Get target type for this column
        target_type = column_types[epic_col]['secu_type']  # Use secuTrial type as the target

        # Compare values for rows with matching (FID, SSR)
        for fid, ssr in matching_keys:
            epic_row = epic_df_copy.loc[(epic_df_copy["FID"] == fid) & (epic_df_copy["SSR"] == ssr)]
            secu_row = secuTrial_df_copy.loc[(secuTrial_df_copy["FID"] == fid) & (secuTrial_df_copy["SSR"] == ssr)]
            
            if epic_row.empty or secu_row.empty:
                continue  # Skip if no matching row found
                
            epic_value = epic_row[epic_col].iloc[0] if epic_col in epic_row.columns else pd.NA
            secu_value = secu_row[secu_col].iloc[0] if secu_col in secu_row.columns else pd.NA
            
            # Both values are NaN/missing - count as match
            if pd.isna(epic_value) and pd.isna(secu_value):
                match_count += 1
                continue
                
            # Only secuTrial value is NaN/missing
            if pd.isna(secu_value) and not pd.isna(epic_value):
                secu_missing_count += 1
                continue
                
            # Only EPIC value is NaN/missing
            if pd.isna(epic_value) and not pd.isna(secu_value):
                epic_missing_count += 1
                continue

            # Compare values using the target type
            if equivalent_values(epic_value, secu_value, target_type):
                match_count += 1
            else:
                mismatch_count += 1
                mismatched_results.append({
                    'FID': fid,
                    'SSR': ssr,
                    'EPIC Column': epic_col,
                    'SecuTrial Column': secu_col,
                    'EPIC Value': str(epic_value),
                    'SecuTrial Value': str(secu_value),
                    'EPIC Expected Type': column_types[epic_col]['epic_type'],
                    'SecuTrial Expected Type': target_type,
                    'EPIC Actual Type': type(epic_value).__name__,
                    'SecuTrial Actual Type': type(secu_value).__name__
                })

    # Calculate percentages
    total_compared = match_count + mismatch_count + secu_missing_count + epic_missing_count
    percentage_stats = {
        "Matching Variables (%)": round((match_count / total_compared) * 100, 2) if total_compared else 0,
        "Variables Missing in EPIC (%)": round((epic_missing_count / total_compared) * 100, 2) if total_compared else 0,
        "Variables Missing in SecuTrial (%)": round((secu_missing_count / total_compared) * 100, 2) if total_compared else 0,
        "Mismatched Variables (%)": round((mismatch_count / total_compared) * 100, 2) if total_compared else 0,
        "Total Comparisons": total_compared,
        "Matches": match_count,
        "EPIC Missing": epic_missing_count,
        "SecuTrial Missing": secu_missing_count,
        "Mismatches": mismatch_count
    }

    return pd.DataFrame(mismatched_results), percentage_stats

In [39]:
def compare_epic_secuTrial(epic_df, secuTrial_df, mapping_df, value_mappings=None):
    """
    Compares values and data types between EPIC and secuTrial DataFrames using a mapping file,
    accounting for data type differences properly. Includes monthly breakdown of statistics.

    Args:
        epic_df (DataFrame): The EPIC dataset.
        secuTrial_df (DataFrame): The SecuTrial dataset.
        mapping_df (DataFrame): The mapping Excel file with data type information.
        value_mappings (dict, optional): Dictionary for value conversions in EPIC.

    Returns:
        DataFrame: DataFrame with mismatched results in values and data types.
        dict: Dictionary containing percentage statistics.
        dict: Dictionary containing monthly statistics.
    """
    
    import pandas as pd
    import re
    from datetime import datetime
    
    if value_mappings is None:
        value_mappings = {}
        
    # Create working copies to avoid modifying original dataframes
    epic_df_copy = epic_df.copy()
    secuTrial_df_copy = secuTrial_df.copy()
    
    # Replace missing value indicators
    epic_df_copy.replace(-9999, pd.NA, inplace=True)
    secuTrial_df_copy.replace(-9999, pd.NA, inplace=True)

    # Ensure necessary columns exist before comparison
    if "FID" not in epic_df_copy.columns or "SSR" not in epic_df_copy.columns:
        raise ValueError("EPIC DataFrame must contain 'FID' and 'SSR' columns.")
    if "FID" not in secuTrial_df_copy.columns or "SSR" not in secuTrial_df_copy.columns:
        raise ValueError("SecuTrial DataFrame must contain 'FID' and 'SSR' columns.")
        
    # Modify line ~40-43 in the function
    if "enct.arrival_date" not in epic_df_copy.columns:
        raise ValueError("EPIC DataFrame must contain 'enct.arrival_date' column for monthly breakdown.")
    if "Arrival at hospital" not in secuTrial_df_copy.columns:
        raise ValueError("SecuTrial DataFrame must contain 'Arrival at hospital' column for monthly breakdown.")

    # And then use these columns for date conversion ~46-47
    epic_df_copy['DATE'] = pd.to_datetime(epic_df_copy['enct.arrival_date'], errors='coerce')
    secuTrial_df_copy['DATE'] = pd.to_datetime(secuTrial_df_copy['Arrival at hospital'], errors='coerce')

    # Create a set of (FID, SSR) pairs that exist in both DataFrames
    matching_keys = set(epic_df_copy[['FID', 'SSR']].apply(tuple, axis=1)) & set(secuTrial_df_copy[['FID', 'SSR']].apply(tuple, axis=1))

    # Store mismatched results
    mismatched_results = []
    match_count = 0
    secu_missing_count = 0
    epic_missing_count = 0
    mismatch_count = 0
    total_comparisons = 0
    
    # Setup monthly statistics tracking
    months = {4: 'April', 5: 'May', 6: 'June', 7: 'July', 8: 'August', 
              9: 'September', 10: 'October', 11: 'November', 12: 'December'}
    
    monthly_stats = {month_name: {'match_count': 0, 'secu_missing_count': 0, 
                                'epic_missing_count': 0, 'mismatch_count': 0, 
                                'total_compared': 0} 
                   for month_name in months.values()}
    
    # Helper function to standardize boolean values
    def standardize_boolean(value):
        if pd.isna(value):
            return pd.NA
        
        if isinstance(value, bool):
            return "yes" if value else "no"
        elif isinstance(value, (int, float)):
            return "yes" if value else "no"
        elif isinstance(value, str):
            if value.lower() in ['true', 'yes', 'y', '1', 't']:
                return "yes"
            elif value.lower() in ['false', 'no', 'n', '0', 'f']:
                return "no"
        
        return str(value)
    
    # Helper function to convert values to the correct type
    def convert_to_type(value, target_type):
        """Convert value to specified type with specific formatting"""
        if pd.isna(value):
            return pd.NA
            
        # Handle various data types
        if not isinstance(target_type, str):
            return value  # If no type specified, return as is
            
        # Check for float with decimal specification (e.g., float-1, float-2)
        float_match = re.match(r'float-(\d+)', target_type.lower())
        if float_match:
            try:
                decimal_places = int(float_match.group(1))
                if value == '':
                    return pd.NA
                float_val = float(value)
                return round(float_val, decimal_places)
            except (ValueError, TypeError):
                return pd.NA
        
        if target_type.lower() in ['int', 'integer', 'int64', 'int32']:
            try:
                return int(float(value)) if value != '' else pd.NA
            except (ValueError, TypeError):
                return pd.NA
        elif target_type.lower() in ['float', 'double', 'numeric', 'float64', 'float32']:
            try:
                return float(value) if value != '' else pd.NA
            except (ValueError, TypeError):
                return pd.NA
        elif target_type.lower() in ['date', 'datetime', 'timestamp']:
            try:
                if value == '':
                    return pd.NA
                # Convert to datetime and then to yyyymmdd hh:mm format
                dt = pd.to_datetime(value)
                return dt.strftime('%Y%m%d %H:%M')
            except (ValueError, TypeError, AttributeError):
                return pd.NA
        elif target_type.lower() in ['bool', 'boolean']:
            return standardize_boolean(value)
        else:
            # Default to string for text, categorical, etc.
            return str(value) if value is not None and value != '' else pd.NA
    
    # Helper function to check if values are equivalent
    def equivalent_values(val1, val2, target_type):
        """Compare values with formatted type awareness"""
        # Handle NaN values consistently
        if pd.isna(val1) and pd.isna(val2):
            return True
        elif pd.isna(val1) or pd.isna(val2):
            return False
            
        # Check for float with decimal specification (e.g., float-1, float-2)
        float_match = re.match(r'float-(\d+)', target_type.lower()) if isinstance(target_type, str) else None
        if float_match:
            try:
                decimal_places = int(float_match.group(1))
                val1_rounded = round(float(val1), decimal_places)
                val2_rounded = round(float(val2), decimal_places)
                return val1_rounded == val2_rounded
            except (ValueError, TypeError):
                return False
        
        # Boolean comparison (standardized to yes/no)
        if isinstance(target_type, str) and target_type.lower() in ['bool', 'boolean']:
            val1_std = standardize_boolean(val1)
            val2_std = standardize_boolean(val2)
            return val1_std == val2_std
        
        # Special handling for numeric types
        if isinstance(val1, (int, float)) and isinstance(val2, (int, float)):
            try:
                return abs(float(val1) - float(val2)) < 1e-6
            except (ValueError, TypeError):
                return False
        
        # Date comparison (already in string format)
        if isinstance(target_type, str) and target_type.lower() in ['date', 'datetime', 'timestamp']:
            return val1 == val2
            
        # String comparison (case insensitive)
        elif isinstance(val1, str) and isinstance(val2, str):
            return val1.strip().lower() == val2.strip().lower()
            
        # Default comparison
        else:
            return str(val1) == str(val2)
    
    # Build a column mapping dictionary for easier lookups
    column_mappings = {}
    column_types = {}
    
    for _, row in mapping_df.iterrows():
        epic_column_name = row.get('EPIC_varColumnName', None)
        secuTrial_column_name = row.get('sT_varColumnName', None)
        epic_dtype = row.get('EPIC_varType', 'text')  # Default to text if not specified
        secuTrial_dtype = row.get('sT_varType', 'text')  # Default to text if not specified
        column_source = row.get('EPIC_exportFileName', None)
        secu_source = row.get('sT_exportFileName', None)

        if not isinstance(epic_column_name, str) or not isinstance(secuTrial_column_name, str):
            continue  # Skip if column names are missing

        # Determine EPIC column prefix
        prefix = ""
        if isinstance(column_source, str):
            if "encounter" in column_source:
                prefix = "enct."
            elif "flowsheet" in column_source:
                prefix = "flow."
            elif "imaging" in column_source:
                prefix = "img."
            elif "lab" in column_source:
                prefix = "lab."
            elif "medication" in column_source:
                prefix = "med."
            elif "monitor" in column_source:
                prefix = "mon."

        # Determine SecuTrial column suffix
        suffix = ""
        if isinstance(secu_source, str) and "REVASC" in secu_source:
            suffix = ".revas"

        # Construct fully qualified column names
        epic_col = f"{prefix}{epic_column_name}"  # EPIC column with prefix
        secu_col = f"{secuTrial_column_name}{suffix}"  # SecuTrial column with suffix
        
        # If secuTrial type is int, override EPIC type to also be int
        if secuTrial_dtype.lower() in ['int', 'integer', 'int64', 'int32']:
            epic_dtype = 'int'
        # If secuTrial type is float or float-n, override EPIC type
        elif secuTrial_dtype.lower() in ['float', 'double', 'numeric', 'float64', 'float32'] or re.match(r'float-\d+', secuTrial_dtype.lower()):
            epic_dtype = secuTrial_dtype
        
        # Store the mapping
        column_mappings[epic_col] = secu_col
        column_types[epic_col] = {'epic_type': epic_dtype, 'secu_type': secuTrial_dtype}
    
    # First, apply value mappings to EPIC data
    for col, mapping in value_mappings.items():
        if col in epic_df_copy.columns:
            epic_df_copy[col] = epic_df_copy[col].map(lambda x: mapping.get(x, x))
    
    # Next, convert data types in both dataframes
    for epic_col, secu_col in column_mappings.items():
        # For EPIC dataframe
        if epic_col in epic_df_copy.columns:
            target_type = column_types[epic_col]['epic_type']
            epic_df_copy[epic_col] = epic_df_copy[epic_col].apply(lambda x: convert_to_type(x, target_type))
        
        # For secuTrial dataframe
        if secu_col in secuTrial_df_copy.columns:
            target_type = column_types[epic_col]['secu_type']
            secuTrial_df_copy[secu_col] = secuTrial_df_copy[secu_col].apply(lambda x: convert_to_type(x, target_type))
    
    # Now compare the values
    for epic_col, secu_col in column_mappings.items():
        # Check if columns exist in respective DataFrames
        if epic_col not in epic_df_copy.columns and secu_col in secuTrial_df_copy.columns:
            epic_missing_count += 1
            continue
        elif secu_col not in secuTrial_df_copy.columns and epic_col in epic_df_copy.columns:
            secu_missing_count += 1
            continue
        elif epic_col not in epic_df_copy.columns and secu_col not in secuTrial_df_copy.columns:
            continue  # Skip comparison if column is missing in both

        total_comparisons += 1
        
        # Get target type for this column
        target_type = column_types[epic_col]['secu_type']  # Use secuTrial type as the target

        # Compare values for rows with matching (FID, SSR)
        for fid, ssr in matching_keys:
            epic_row = epic_df_copy.loc[(epic_df_copy["FID"] == fid) & (epic_df_copy["SSR"] == ssr)]
            secu_row = secuTrial_df_copy.loc[(secuTrial_df_copy["FID"] == fid) & (secuTrial_df_copy["SSR"] == ssr)]
            
            if epic_row.empty or secu_row.empty:
                continue  # Skip if no matching row found
                
            epic_value = epic_row[epic_col].iloc[0] if epic_col in epic_row.columns else pd.NA
            secu_value = secu_row[secu_col].iloc[0] if secu_col in secu_row.columns else pd.NA
            
            # Get the month for this record (use epic date if available, else secu date)
            record_date = None
            if not epic_row.empty and 'DATE' in epic_row.columns and not pd.isna(epic_row['DATE'].iloc[0]):
                record_date = epic_row['DATE'].iloc[0]
            elif not secu_row.empty and 'DATE' in secu_row.columns and not pd.isna(secu_row['DATE'].iloc[0]):
                record_date = secu_row['DATE'].iloc[0]
                
            # Skip if no valid date or not in April-December range
            if record_date is None:
                continue
                
            record_month = record_date.month
            # Skip if not in our target month range (April-December)
            if record_month < 4 or record_month > 12:
                continue
                
            month_name = months[record_month]
                
            # Both values are NaN/missing - count as match
            if pd.isna(epic_value) and pd.isna(secu_value):
                match_count += 1
                monthly_stats[month_name]['match_count'] += 1
                monthly_stats[month_name]['total_compared'] += 1
                continue
                
            # Only secuTrial value is NaN/missing
            if pd.isna(secu_value) and not pd.isna(epic_value):
                secu_missing_count += 1
                monthly_stats[month_name]['secu_missing_count'] += 1
                monthly_stats[month_name]['total_compared'] += 1
                continue
                
            # Only EPIC value is NaN/missing
            if pd.isna(epic_value) and not pd.isna(secu_value):
                epic_missing_count += 1
                monthly_stats[month_name]['epic_missing_count'] += 1
                monthly_stats[month_name]['total_compared'] += 1
                continue

            # Compare values using the target type
            if equivalent_values(epic_value, secu_value, target_type):
                match_count += 1
                monthly_stats[month_name]['match_count'] += 1
            else:
                mismatch_count += 1
                monthly_stats[month_name]['mismatch_count'] += 1
                mismatched_results.append({
                    'FID': fid,
                    'SSR': ssr,
                    'Month': month_name,
                    'DATE': record_date,
                    'EPIC Column': epic_col,
                    'SecuTrial Column': secu_col,
                    'EPIC Value': str(epic_value),
                    'SecuTrial Value': str(secu_value),
                    'EPIC Expected Type': column_types[epic_col]['epic_type'],
                    'SecuTrial Expected Type': target_type,
                    'EPIC Actual Type': type(epic_value).__name__,
                    'SecuTrial Actual Type': type(secu_value).__name__
                })
            
            monthly_stats[month_name]['total_compared'] += 1

    # Calculate percentages
    total_compared = match_count + mismatch_count + secu_missing_count + epic_missing_count
    percentage_stats = {
        "Matching Variables (%)": round((match_count / total_compared) * 100, 2) if total_compared else 0,
        "Variables Missing in EPIC (%)": round((epic_missing_count / total_compared) * 100, 2) if total_compared else 0,
        "Variables Missing in SecuTrial (%)": round((secu_missing_count / total_compared) * 100, 2) if total_compared else 0,
        "Mismatched Variables (%)": round((mismatch_count / total_compared) * 100, 2) if total_compared else 0,
        "Total Comparisons": total_compared,
        "Matches": match_count,
        "EPIC Missing": epic_missing_count,
        "SecuTrial Missing": secu_missing_count,
        "Mismatches": mismatch_count
    }
    
    # Calculate monthly percentages
    monthly_percentage_stats = {}
    for month, stats in monthly_stats.items():
        total = stats['total_compared']
        if total > 0:
            monthly_percentage_stats[month] = {
                "Matching Variables (%)": round((stats['match_count'] / total) * 100, 2),
                "Variables Missing in EPIC (%)": round((stats['epic_missing_count'] / total) * 100, 2),
                "Variables Missing in SecuTrial (%)": round((stats['secu_missing_count'] / total) * 100, 2),
                "Mismatched Variables (%)": round((stats['mismatch_count'] / total) * 100, 2),
                "Total Comparisons": total,
                "Matches": stats['match_count'],
                "EPIC Missing": stats['epic_missing_count'],
                "SecuTrial Missing": stats['secu_missing_count'],
                "Mismatches": stats['mismatch_count']
            }
        else:
            monthly_percentage_stats[month] = {
                "Matching Variables (%)": 0,
                "Variables Missing in EPIC (%)": 0,
                "Variables Missing in SecuTrial (%)": 0,
                "Mismatched Variables (%)": 0,
                "Total Comparisons": 0,
                "Matches": 0,
                "EPIC Missing": 0,
                "SecuTrial Missing": 0,
                "Mismatches": 0
            }

    return pd.DataFrame(mismatched_results), percentage_stats, monthly_percentage_stats

In [40]:
def compare_epic_secuTrial(epic_df, secuTrial_df, mapping_df, value_mappings=None):
    """
    Compares values and data types between EPIC and secuTrial DataFrames using a mapping file,
    accounting for data type differences properly. Includes monthly breakdown of statistics
    and variable-level statistics.

    Args:
        epic_df (DataFrame): The EPIC dataset.
        secuTrial_df (DataFrame): The SecuTrial dataset.
        mapping_df (DataFrame): The mapping Excel file with data type information.
        value_mappings (dict, optional): Dictionary for value conversions in EPIC.

    Returns:
        DataFrame: DataFrame with mismatched results in values and data types.
        dict: Dictionary containing percentage statistics.
        dict: Dictionary containing monthly statistics.
        dict: Dictionary containing variable-level statistics.
    """
    
    import pandas as pd
    import re
    from datetime import datetime
    
    if value_mappings is None:
        value_mappings = {}
        
    # Create working copies to avoid modifying original dataframes
    epic_df_copy = epic_df.copy()
    secuTrial_df_copy = secuTrial_df.copy()
    
    # Replace missing value indicators
    epic_df_copy.replace(-9999, pd.NA, inplace=True)
    secuTrial_df_copy.replace(-9999, pd.NA, inplace=True)

    # Ensure necessary columns exist before comparison
    if "FID" not in epic_df_copy.columns or "SSR" not in epic_df_copy.columns:
        raise ValueError("EPIC DataFrame must contain 'FID' and 'SSR' columns.")
    if "FID" not in secuTrial_df_copy.columns or "SSR" not in secuTrial_df_copy.columns:
        raise ValueError("SecuTrial DataFrame must contain 'FID' and 'SSR' columns.")
        
    # Modify line ~40-43 in the function
    if "enct.arrival_date" not in epic_df_copy.columns:
        raise ValueError("EPIC DataFrame must contain 'enct.arrival_date' column for monthly breakdown.")
    if "Arrival at hospital" not in secuTrial_df_copy.columns:
        raise ValueError("SecuTrial DataFrame must contain 'Arrival at hospital' column for monthly breakdown.")

    # And then use these columns for date conversion ~46-47
    epic_df_copy['DATE'] = pd.to_datetime(epic_df_copy['enct.arrival_date'], errors='coerce')
    secuTrial_df_copy['DATE'] = pd.to_datetime(secuTrial_df_copy['Arrival at hospital'], errors='coerce')

    # Create a set of (FID, SSR) pairs that exist in both DataFrames
    matching_keys = set(epic_df_copy[['FID', 'SSR']].apply(tuple, axis=1)) & set(secuTrial_df_copy[['FID', 'SSR']].apply(tuple, axis=1))

    # Store mismatched results
    mismatched_results = []
    match_count = 0
    secu_missing_count = 0
    epic_missing_count = 0
    mismatch_count = 0
    total_comparisons = 0
    
    # Setup monthly statistics tracking
    months = {4: 'April', 5: 'May', 6: 'June', 7: 'July', 8: 'August', 
              9: 'September', 10: 'October', 11: 'November', 12: 'December'}
    
    monthly_stats = {month_name: {'match_count': 0, 'secu_missing_count': 0, 
                                'epic_missing_count': 0, 'mismatch_count': 0, 
                                'total_compared': 0} 
                   for month_name in months.values()}
    
    # Create a dictionary to store variable-level statistics
    variable_stats = {}
    
    # Helper function to standardize boolean values
    def standardize_boolean(value):
        if pd.isna(value):
            return pd.NA
        
        if isinstance(value, bool):
            return "yes" if value else "no"
        elif isinstance(value, (int, float)):
            return "yes" if value else "no"
        elif isinstance(value, str):
            if value.lower() in ['true', 'yes', 'y', '1', 't']:
                return "yes"
            elif value.lower() in ['false', 'no', 'n', '0', 'f']:
                return "no"
        
        return str(value)
    
    # Helper function to convert values to the correct type
    def convert_to_type(value, target_type):
        """Convert value to specified type with specific formatting"""
        if pd.isna(value):
            return pd.NA
            
        # Handle various data types
        if not isinstance(target_type, str):
            return value  # If no type specified, return as is
            
        # Check for float with decimal specification (e.g., float-1, float-2)
        float_match = re.match(r'float-(\d+)', target_type.lower())
        if float_match:
            try:
                decimal_places = int(float_match.group(1))
                if value == '':
                    return pd.NA
                float_val = float(value)
                return round(float_val, decimal_places)
            except (ValueError, TypeError):
                return pd.NA
        
        if target_type.lower() in ['int', 'integer', 'int64', 'int32']:
            try:
                return int(float(value)) if value != '' else pd.NA
            except (ValueError, TypeError):
                return pd.NA
        elif target_type.lower() in ['float', 'double', 'numeric', 'float64', 'float32']:
            try:
                return float(value) if value != '' else pd.NA
            except (ValueError, TypeError):
                return pd.NA
        elif target_type.lower() in ['date', 'datetime', 'timestamp']:
            try:
                if value == '':
                    return pd.NA
                # Convert to datetime and then to yyyymmdd hh:mm format
                dt = pd.to_datetime(value)
                return dt.strftime('%Y%m%d %H:%M')
            except (ValueError, TypeError, AttributeError):
                return pd.NA
        elif target_type.lower() in ['bool', 'boolean']:
            return standardize_boolean(value)
        else:
            # Default to string for text, categorical, etc.
            return str(value) if value is not None and value != '' else pd.NA
    
    # Helper function to check if values are equivalent
    def equivalent_values(val1, val2, target_type):
        """Compare values with formatted type awareness"""
        # Handle NaN values consistently
        if pd.isna(val1) and pd.isna(val2):
            return True
        elif pd.isna(val1) or pd.isna(val2):
            return False
            
        # Check for float with decimal specification (e.g., float-1, float-2)
        float_match = re.match(r'float-(\d+)', target_type.lower()) if isinstance(target_type, str) else None
        if float_match:
            try:
                decimal_places = int(float_match.group(1))
                val1_rounded = round(float(val1), decimal_places)
                val2_rounded = round(float(val2), decimal_places)
                return val1_rounded == val2_rounded
            except (ValueError, TypeError):
                return False
        
        # Boolean comparison (standardized to yes/no)
        if isinstance(target_type, str) and target_type.lower() in ['bool', 'boolean']:
            val1_std = standardize_boolean(val1)
            val2_std = standardize_boolean(val2)
            return val1_std == val2_std
        
        # Special handling for numeric types
        if isinstance(val1, (int, float)) and isinstance(val2, (int, float)):
            try:
                return abs(float(val1) - float(val2)) < 1e-6
            except (ValueError, TypeError):
                return False
        
        # Date comparison (already in string format)
        if isinstance(target_type, str) and target_type.lower() in ['date', 'datetime', 'timestamp']:
            return val1 == val2
            
        # String comparison (case insensitive)
        elif isinstance(val1, str) and isinstance(val2, str):
            return val1.strip().lower() == val2.strip().lower()
            
        # Default comparison
        else:
            return str(val1) == str(val2)
    
    # Build a column mapping dictionary for easier lookups
    column_mappings = {}
    column_types = {}
    
    for _, row in mapping_df.iterrows():
        epic_column_name = row.get('EPIC_varColumnName', None)
        secuTrial_column_name = row.get('sT_varColumnName', None)
        epic_dtype = row.get('EPIC_varType', 'text')  # Default to text if not specified
        secuTrial_dtype = row.get('sT_varType', 'text')  # Default to text if not specified
        column_source = row.get('EPIC_exportFileName', None)
        secu_source = row.get('sT_exportFileName', None)

        if not isinstance(epic_column_name, str) or not isinstance(secuTrial_column_name, str):
            continue  # Skip if column names are missing

        # Determine EPIC column prefix
        prefix = ""
        if isinstance(column_source, str):
            if "encounter" in column_source:
                prefix = "enct."
            elif "flowsheet" in column_source:
                prefix = "flow."
            elif "imaging" in column_source:
                prefix = "img."
            elif "lab" in column_source:
                prefix = "lab."
            elif "medication" in column_source:
                prefix = "med."
            elif "monitor" in column_source:
                prefix = "mon."

        # Determine SecuTrial column suffix
        suffix = ""
        if isinstance(secu_source, str) and "REVASC" in secu_source:
            suffix = ".revas"

        # Construct fully qualified column names
        epic_col = f"{prefix}{epic_column_name}"  # EPIC column with prefix
        secu_col = f"{secuTrial_column_name}{suffix}"  # SecuTrial column with suffix
        
        # If secuTrial type is int, override EPIC type to also be int
        if secuTrial_dtype.lower() in ['int', 'integer', 'int64', 'int32']:
            epic_dtype = 'int'
        # If secuTrial type is float or float-n, override EPIC type
        elif secuTrial_dtype.lower() in ['float', 'double', 'numeric', 'float64', 'float32'] or re.match(r'float-\d+', secuTrial_dtype.lower()):
            epic_dtype = secuTrial_dtype
        
        # Store the mapping
        column_mappings[epic_col] = secu_col
        column_types[epic_col] = {'epic_type': epic_dtype, 'secu_type': secuTrial_dtype}
        
        # Initialize variable stats for this column pair
        variable_stats[f"{epic_col} <-> {secu_col}"] = {
            'match_count': 0,
            'epic_missing_count': 0,
            'secu_missing_count': 0,
            'mismatch_count': 0,
            'total_compared': 0,
            'epic_type': epic_dtype,
            'secu_type': secuTrial_dtype
        }
    
    # First, apply value mappings to EPIC data
    for col, mapping in value_mappings.items():
        if col in epic_df_copy.columns:
            epic_df_copy[col] = epic_df_copy[col].map(lambda x: mapping.get(x, x))
    
    # Next, convert data types in both dataframes
    for epic_col, secu_col in column_mappings.items():
        # For EPIC dataframe
        if epic_col in epic_df_copy.columns:
            target_type = column_types[epic_col]['epic_type']
            epic_df_copy[epic_col] = epic_df_copy[epic_col].apply(lambda x: convert_to_type(x, target_type))
        
        # For secuTrial dataframe
        if secu_col in secuTrial_df_copy.columns:
            target_type = column_types[epic_col]['secu_type']
            secuTrial_df_copy[secu_col] = secuTrial_df_copy[secu_col].apply(lambda x: convert_to_type(x, target_type))
    
    # Now compare the values
    for epic_col, secu_col in column_mappings.items():
        var_key = f"{epic_col} <-> {secu_col}"
        
        # Check if columns exist in respective DataFrames
        if epic_col not in epic_df_copy.columns and secu_col in secuTrial_df_copy.columns:
            epic_missing_count += 1
            variable_stats[var_key]['epic_missing_count'] += 1
            variable_stats[var_key]['total_compared'] += 1
            continue
        elif secu_col not in secuTrial_df_copy.columns and epic_col in epic_df_copy.columns:
            secu_missing_count += 1
            variable_stats[var_key]['secu_missing_count'] += 1
            variable_stats[var_key]['total_compared'] += 1
            continue
        elif epic_col not in epic_df_copy.columns and secu_col not in secuTrial_df_copy.columns:
            continue  # Skip comparison if column is missing in both

        total_comparisons += 1
        
        # Get target type for this column
        target_type = column_types[epic_col]['secu_type']  # Use secuTrial type as the target

        # Compare values for rows with matching (FID, SSR)
        for fid, ssr in matching_keys:
            epic_row = epic_df_copy.loc[(epic_df_copy["FID"] == fid) & (epic_df_copy["SSR"] == ssr)]
            secu_row = secuTrial_df_copy.loc[(secuTrial_df_copy["FID"] == fid) & (secuTrial_df_copy["SSR"] == ssr)]
            
            if epic_row.empty or secu_row.empty:
                continue  # Skip if no matching row found
                
            epic_value = epic_row[epic_col].iloc[0] if epic_col in epic_row.columns else pd.NA
            secu_value = secu_row[secu_col].iloc[0] if secu_col in secu_row.columns else pd.NA
            
            # Get the month for this record (use epic date if available, else secu date)
            record_date = None
            if not epic_row.empty and 'DATE' in epic_row.columns and not pd.isna(epic_row['DATE'].iloc[0]):
                record_date = epic_row['DATE'].iloc[0]
            elif not secu_row.empty and 'DATE' in secu_row.columns and not pd.isna(secu_row['DATE'].iloc[0]):
                record_date = secu_row['DATE'].iloc[0]
                
            # Skip if no valid date or not in April-December range
            if record_date is None:
                continue
                
            record_month = record_date.month
            # Skip if not in our target month range (April-December)
            if record_month < 4 or record_month > 12:
                continue
                
            month_name = months[record_month]
                
            # Both values are NaN/missing - count as match
            if pd.isna(epic_value) and pd.isna(secu_value):
                match_count += 1
                monthly_stats[month_name]['match_count'] += 1
                monthly_stats[month_name]['total_compared'] += 1
                variable_stats[var_key]['match_count'] += 1
                variable_stats[var_key]['total_compared'] += 1
                continue
                
            # Only secuTrial value is NaN/missing
            if pd.isna(secu_value) and not pd.isna(epic_value):
                secu_missing_count += 1
                monthly_stats[month_name]['secu_missing_count'] += 1
                monthly_stats[month_name]['total_compared'] += 1
                variable_stats[var_key]['secu_missing_count'] += 1
                variable_stats[var_key]['total_compared'] += 1
                continue
                
            # Only EPIC value is NaN/missing
            if pd.isna(epic_value) and not pd.isna(secu_value):
                epic_missing_count += 1
                monthly_stats[month_name]['epic_missing_count'] += 1
                monthly_stats[month_name]['total_compared'] += 1
                variable_stats[var_key]['epic_missing_count'] += 1
                variable_stats[var_key]['total_compared'] += 1
                continue

            # Compare values using the target type
            if equivalent_values(epic_value, secu_value, target_type):
                match_count += 1
                monthly_stats[month_name]['match_count'] += 1
                variable_stats[var_key]['match_count'] += 1
            else:
                mismatch_count += 1
                monthly_stats[month_name]['mismatch_count'] += 1
                variable_stats[var_key]['mismatch_count'] += 1
                mismatched_results.append({
                    'FID': fid,
                    'SSR': ssr,
                    'Month': month_name,
                    'DATE': record_date,
                    'EPIC Column': epic_col,
                    'SecuTrial Column': secu_col,
                    'EPIC Value': str(epic_value),
                    'SecuTrial Value': str(secu_value),
                    'EPIC Expected Type': column_types[epic_col]['epic_type'],
                    'SecuTrial Expected Type': target_type,
                    'EPIC Actual Type': type(epic_value).__name__,
                    'SecuTrial Actual Type': type(secu_value).__name__
                })
            
            monthly_stats[month_name]['total_compared'] += 1
            variable_stats[var_key]['total_compared'] += 1

    # Calculate percentages
    total_compared = match_count + mismatch_count + secu_missing_count + epic_missing_count
    percentage_stats = {
        "Matching Variables (%)": round((match_count / total_compared) * 100, 2) if total_compared else 0,
        "Variables Missing in EPIC (%)": round((epic_missing_count / total_compared) * 100, 2) if total_compared else 0,
        "Variables Missing in SecuTrial (%)": round((secu_missing_count / total_compared) * 100, 2) if total_compared else 0,
        "Mismatched Variables (%)": round((mismatch_count / total_compared) * 100, 2) if total_compared else 0,
        "Total Comparisons": total_compared,
        "Matches": match_count,
        "EPIC Missing": epic_missing_count,
        "SecuTrial Missing": secu_missing_count,
        "Mismatches": mismatch_count
    }
    
    # Calculate monthly percentages
    monthly_percentage_stats = {}
    for month, stats in monthly_stats.items():
        total = stats['total_compared']
        if total > 0:
            monthly_percentage_stats[month] = {
                "Matching Variables (%)": round((stats['match_count'] / total) * 100, 2),
                "Variables Missing in EPIC (%)": round((stats['epic_missing_count'] / total) * 100, 2),
                "Variables Missing in SecuTrial (%)": round((stats['secu_missing_count'] / total) * 100, 2),
                "Mismatched Variables (%)": round((stats['mismatch_count'] / total) * 100, 2),
                "Total Comparisons": total,
                "Matches": stats['match_count'],
                "EPIC Missing": stats['epic_missing_count'],
                "SecuTrial Missing": stats['secu_missing_count'],
                "Mismatches": stats['mismatch_count']
            }
        else:
            monthly_percentage_stats[month] = {
                "Matching Variables (%)": 0,
                "Variables Missing in EPIC (%)": 0,
                "Variables Missing in SecuTrial (%)": 0,
                "Mismatched Variables (%)": 0,
                "Total Comparisons": 0,
                "Matches": 0,
                "EPIC Missing": 0,
                "SecuTrial Missing": 0,
                "Mismatches": 0
            }
    
    # Calculate variable-level percentages
    variable_percentage_stats = {}
    for var_key, stats in variable_stats.items():
        total = stats['total_compared']
        if total > 0:
            variable_percentage_stats[var_key] = {
                "Matching Values (%)": round((stats['match_count'] / total) * 100, 2),
                "Values Missing in EPIC (%)": round((stats['epic_missing_count'] / total) * 100, 2),
                "Values Missing in SecuTrial (%)": round((stats['secu_missing_count'] / total) * 100, 2),
                "Mismatched Values (%)": round((stats['mismatch_count'] / total) * 100, 2),
                "Total Comparisons": total,
                "Matches": stats['match_count'],
                "EPIC Missing": stats['epic_missing_count'],
                "SecuTrial Missing": stats['secu_missing_count'],
                "Mismatches": stats['mismatch_count'],
                "EPIC Type": stats['epic_type'],
                "SecuTrial Type": stats['secu_type']
            }
        else:
            variable_percentage_stats[var_key] = {
                "Matching Values (%)": 0,
                "Values Missing in EPIC (%)": 0,
                "Values Missing in SecuTrial (%)": 0,
                "Mismatched Values (%)": 0,
                "Total Comparisons": 0,
                "Matches": 0,
                "EPIC Missing": 0,
                "SecuTrial Missing": 0,
                "Mismatches": 0,
                "EPIC Type": stats['epic_type'],
                "SecuTrial Type": stats['secu_type']
            }

    return pd.DataFrame(mismatched_results), percentage_stats, monthly_percentage_stats, variable_percentage_stats

# Function to get top problematic variables
def get_top_problematic_variables(variable_stats, sort_by='mismatch_percent', top_n=10):
    """
    Identify the most problematic variables based on specified criteria
    
    Args:
        variable_stats (dict): Dictionary containing variable-level statistics
        sort_by (str): Criteria to sort by: 'mismatch_percent', 'missing_epic_percent', 
                       'missing_secuTrial_percent', or 'total_problems'
        top_n (int): Number of variables to return
        
    Returns:
        DataFrame: Top problematic variables sorted by the specified criteria
    """
    import pandas as pd
    
    # Create a DataFrame from the variable stats
    var_df = pd.DataFrame()
    
    for var_name, stats in variable_stats.items():
        if stats['Total Comparisons'] > 0:  # Only include variables with actual comparisons
            row = {
                'Variable': var_name,
                'Total Comparisons': stats['Total Comparisons'],
                'Match Count': stats['Matches'],
                'Match Percent': stats['Matching Values (%)'],
                'Mismatch Count': stats['Mismatches'],
                'Mismatch Percent': stats['Mismatched Values (%)'],
                'EPIC Missing Count': stats['EPIC Missing'],
                'EPIC Missing Percent': stats['Values Missing in EPIC (%)'],
                'SecuTrial Missing Count': stats['SecuTrial Missing'],
                'SecuTrial Missing Percent': stats['Values Missing in SecuTrial (%)'],
                'EPIC Type': stats['EPIC Type'],
                'SecuTrial Type': stats['SecuTrial Type'],
                'Total Problems': stats['Mismatches'] + stats['EPIC Missing'] + stats['SecuTrial Missing'],
                'Total Problem Percent': (100 - stats['Matching Values (%)'])
            }
            var_df = pd.concat([var_df, pd.DataFrame([row])], ignore_index=True)
    
    # Sort based on criteria
    if sort_by == 'mismatch_percent':
        var_df = var_df.sort_values(by='Mismatch Percent', ascending=False)
    elif sort_by == 'missing_epic_percent':
        var_df = var_df.sort_values(by='EPIC Missing Percent', ascending=False)
    elif sort_by == 'missing_secuTrial_percent':
        var_df = var_df.sort_values(by='SecuTrial Missing Percent', ascending=False)
    elif sort_by == 'total_problems':
        var_df = var_df.sort_values(by='Total Problem Percent', ascending=False)
    else:
        var_df = var_df.sort_values(by='Total Problem Percent', ascending=False)
    
    return var_df.head(top_n)

# Function to generate detailed report
def generate_comparison_report(mismatched_results, overall_stats, monthly_stats, variable_stats):
    """
    Generates a comprehensive report from the comparison results
    
    Args:
        mismatched_results (DataFrame): DataFrame with details of mismatched values
        overall_stats (dict): Dictionary with overall statistics
        monthly_stats (dict): Dictionary with monthly breakdown of statistics
        variable_stats (dict): Dictionary with variable-level statistics
        
    Returns:
        str: Markdown formatted report
    """
    import pandas as pd
    from io import StringIO
    
    report = StringIO()
    
    # Overall Summary
    report.write("# EPIC-SecuTrial Data Comparison Summary\n\n")
    report.write("## Overall Statistics\n\n")
    report.write(f"* Total Comparisons: {overall_stats['Total Comparisons']}\n")
    report.write(f"* Matching Variables: {overall_stats['Matches']} ({overall_stats['Matching Variables (%)']}%)\n")
    report.write(f"* Mismatched Variables: {overall_stats['Mismatches']} ({overall_stats['Mismatched Variables (%)']}%)\n")
    report.write(f"* Variables Missing in EPIC: {overall_stats['EPIC Missing']} ({overall_stats['Variables Missing in EPIC (%)']}%)\n")
    report.write(f"* Variables Missing in SecuTrial: {overall_stats['SecuTrial Missing']} ({overall_stats['Variables Missing in SecuTrial (%)']}%)\n\n")
    
    # Monthly Breakdown
    report.write("## Monthly Statistics\n\n")
    monthly_df = pd.DataFrame(columns=['Month', 'Total Comparisons', 'Matching (%)', 'Mismatched (%)', 
                                       'EPIC Missing (%)', 'SecuTrial Missing (%)'])
    
    for month, stats in monthly_stats.items():
        monthly_df = pd.concat([monthly_df, pd.DataFrame([{
            'Month': month,
            'Total Comparisons': stats['Total Comparisons'],
            'Matching (%)': stats['Matching Variables (%)'],
            'Mismatched (%)': stats['Mismatched Variables (%)'],
            'EPIC Missing (%)': stats['Variables Missing in EPIC (%)'],
            'SecuTrial Missing (%)': stats['Variables Missing in SecuTrial (%)']
        }])], ignore_index=True)
    
    report.write(monthly_df.to_markdown(index=False))
    report.write("\n\n")
    
    # Top Problematic Variables
    report.write("## Top 10 Problematic Variables\n\n")
    top_vars = get_top_problematic_variables(variable_stats, sort_by='total_problems', top_n=10)
    report.write(top_vars[['Variable', 'Total Comparisons', 'Match Percent', 'Mismatch Percent', 
                           'EPIC Missing Percent', 'SecuTrial Missing Percent', 'EPIC Type', 'SecuTrial Type']]
                .to_markdown(index=False))
    report.write("\n\n")
    
    # Variables with Type Mismatches
    report.write("## Variables with Type Mismatches\n\n")
    type_mismatches = []
    for var_name, stats in variable_stats.items():
        if stats['EPIC Type'] != stats['SecuTrial Type']:
            type_mismatches.append({
                'Variable': var_name,
                'EPIC Type': stats['EPIC Type'],
                'SecuTrial Type': stats['SecuTrial Type'],
                'Mismatch Percent': stats['Mismatched Values (%)']
            })
    
    if type_mismatches:
        type_mismatch_df = pd.DataFrame(type_mismatches)
        report.write(type_mismatch_df.sort_values(by='Mismatch Percent', ascending=False).to_markdown(index=False))
    else:
        report.write("No type mismatches found.\n")
    
    return report.getvalue()

##### comparison application

In [41]:
# Add DATE column to EPIC dataframe
df_EPIC_all_2['DATE'] = df_EPIC_all_2['enct.arrival_date'] 

# Add DATE column to secuTrial dataframe
df_secuTrial_w_REVAS_2['DATE'] = df_secuTrial_w_REVAS_2['Arrival at hospital']

/var/folders/3_/7d7jg9x11fb6sn0rshxtgnj80000gn/T/ipykernel_18816/3811210246.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_EPIC_all_2['DATE'] = df_EPIC_all_2['enct.arrival_date']


In [42]:
# Now proceed with comparison using the aligned data
#mismatched_results, comparison_stats, monthly_percentage_stats = compare_epic_secuTrial(df_EPIC_all_2, df_secuTrial_w_REVAS_2, df_mapping, value_mappings)

In [43]:
# Now proceed with comparison using the aligned data
mismatched_results, comparison_stats, monthly_percentage_stats, variable_stats = compare_epic_secuTrial(df_EPIC_all_2, df_secuTrial_w_REVAS_2, df_mapping, value_mappings)

# Optionally generate a comprehensive report
report = generate_comparison_report(mismatched_results, comparison_stats, monthly_percentage_stats, variable_stats)
print(report)  # Or save to a file with: with open('comparison_report.md', 'w') as f: f.write(report)

# Export variable statistics to Excel
import pandas as pd

# Create a DataFrame for variable statistics
variable_df = pd.DataFrame()
for var_name, stats in variable_stats.items():
    row = {
        'Variable': var_name,
        'Total Comparisons': stats['Total Comparisons'],
        'Matches': stats['Matches'],
        'Match Percent': stats['Matching Values (%)'],
        'Mismatches': stats['Mismatches'],
        'Mismatch Percent': stats['Mismatched Values (%)'],
        'EPIC Missing': stats['EPIC Missing'],
        'EPIC Missing Percent': stats['Values Missing in EPIC (%)'],
        'SecuTrial Missing': stats['SecuTrial Missing'],
        'SecuTrial Missing Percent': stats['Values Missing in SecuTrial (%)'],
        'EPIC Type': stats['EPIC Type'],
        'SecuTrial Type': stats['SecuTrial Type']
    }
    variable_df = pd.concat([variable_df, pd.DataFrame([row])], ignore_index=True)

# Sort by highest mismatch percentage
variable_df = variable_df.sort_values(by='Mismatch Percent', ascending=False)

# Export to Excel
variable_df.to_excel('variable_statistics.xlsx', index=False)

# You can also export the mismatched results for detailed examination
mismatched_results.to_excel('mismatched_values.xlsx', index=False)

# Export monthly statistics to Excel
monthly_df = pd.DataFrame()
for month, stats in monthly_percentage_stats.items():
    row = {
        'Month': month,
        'Total Comparisons': stats['Total Comparisons'],
        'Matches': stats['Matches'],
        'Match Percent': stats['Matching Variables (%)'],
        'Mismatches': stats['Mismatches'],
        'Mismatch Percent': stats['Mismatched Variables (%)'],
        'EPIC Missing': stats['EPIC Missing'],
        'EPIC Missing Percent': stats['Variables Missing in EPIC (%)'],
        'SecuTrial Missing': stats['SecuTrial Missing'], 
        'SecuTrial Missing Percent': stats['Variables Missing in SecuTrial (%)']
    }
    monthly_df = pd.concat([monthly_df, pd.DataFrame([row])], ignore_index=True)

# Export monthly statistics
with pd.ExcelWriter('comparison_statistics.xlsx') as writer:
    variable_df.to_excel(writer, sheet_name='Variable Statistics', index=False)
    monthly_df.to_excel(writer, sheet_name='Monthly Statistics', index=False)
    pd.DataFrame([comparison_stats]).to_excel(writer, sheet_name='Overall Statistics', index=False)

/var/folders/3_/7d7jg9x11fb6sn0rshxtgnj80000gn/T/ipykernel_18816/2695550548.py:128: UserWarning: Parsing dates in %d.%m.%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  dt = pd.to_datetime(value)
/var/folders/3_/7d7jg9x11fb6sn0rshxtgnj80000gn/T/ipykernel_18816/2695550548.py:533: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  monthly_df = pd.concat([monthly_df, pd.DataFrame([{


# EPIC-SecuTrial Data Comparison Summary

## Overall Statistics

* Total Comparisons: 109448
* Matching Variables: 59859 (54.69%)
* Mismatched Variables: 4101 (3.75%)
* Variables Missing in EPIC: 8362 (7.64%)
* Variables Missing in SecuTrial: 37126 (33.92%)

## Monthly Statistics

| Month     |   Total Comparisons |   Matching (%) |   Mismatched (%) |   EPIC Missing (%) |   SecuTrial Missing (%) |
|:----------|--------------------:|---------------:|-----------------:|-------------------:|------------------------:|
| April     |               10000 |          47.92 |             4.58 |              21.07 |                   26.43 |
| May       |               11760 |          62.86 |             7.99 |              18.78 |                   10.37 |
| June      |                9760 |          67.72 |             4.53 |               3.56 |                   24.2  |
| July      |               11840 |          64.77 |             6.24 |              13.29 |                   15.69 |
| Au

In [44]:
# Convert comparison statistics to a DataFrame and display
comparison_stats_df = pd.DataFrame([comparison_stats])


In [45]:
comparison_stats

{'Matching Variables (%)': 54.69,
 'Variables Missing in EPIC (%)': 7.64,
 'Variables Missing in SecuTrial (%)': 33.92,
 'Mismatched Variables (%)': 3.75,
 'Total Comparisons': 109448,
 'Matches': 59859,
 'EPIC Missing': 8362,
 'SecuTrial Missing': 37126,
 'Mismatches': 4101}

In [46]:
# Save the monthly statistics to Excel
output_dir = base_dir / 'EPIC-export-validation/validation-files'
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
monthly_stats_path = output_dir / f"monthly_validation_stats_{timestamp}.xlsx"

# Convert monthly stats to DataFrames
monthly_stats_df = pd.DataFrame.from_dict(monthly_percentage_stats, orient='index')

# Create a styled Excel writer
with pd.ExcelWriter(monthly_stats_path, engine='openpyxl') as writer:
    # Write monthly stats
    monthly_stats_df.to_excel(writer, sheet_name="Monthly_Stats")
    
    # Write overall stats
    pd.DataFrame([comparison_stats]).to_excel(writer, sheet_name="Overall_Stats", index=False)

print(f"Monthly statistics saved to: {monthly_stats_path}")

# Print summary of monthly stats
print("\nMonthly Statistics Summary:")
print("---------------------------")
for month, stats in monthly_percentage_stats.items():
    print(f"\n{month}:")
    print(f"  Matches: {stats['Matches']} ({stats['Matching Variables (%)']}%)")
    print(f"  Mismatches: {stats['Mismatches']} ({stats['Mismatched Variables (%)']}%)")
    print(f"  Total comparisons: {stats['Total Comparisons']}")

Monthly statistics saved to: /Users/yaskhanloo/Developer/bern-storke-center/EPIC-export-validation/validation-files/monthly_validation_stats_20250414_095129.xlsx

Monthly Statistics Summary:
---------------------------

April:
  Matches: 4792 (47.92%)
  Mismatches: 458 (4.58%)
  Total comparisons: 10000

May:
  Matches: 7392 (62.86%)
  Mismatches: 940 (7.99%)
  Total comparisons: 11760

June:
  Matches: 6609 (67.72%)
  Mismatches: 442 (4.53%)
  Total comparisons: 9760

July:
  Matches: 7669 (64.77%)
  Mismatches: 739 (6.24%)
  Total comparisons: 11840

August:
  Matches: 8050 (67.53%)
  Mismatches: 568 (4.77%)
  Total comparisons: 11920

September:
  Matches: 6428 (49.91%)
  Mismatches: 670 (5.2%)
  Total comparisons: 12880

October:
  Matches: 6746 (47.91%)
  Mismatches: 122 (0.87%)
  Total comparisons: 14080

November:
  Matches: 6466 (45.15%)
  Mismatches: 88 (0.61%)
  Total comparisons: 14320

December:
  Matches: 5707 (44.31%)
  Mismatches: 74 (0.57%)
  Total comparisons: 12880


### Mismatch report generation

In [47]:
def restructure_mismatched_data(differences_df, epic_df):
    """
    Restructures the mismatched data so that each row represents a single (FID, SSR),
    and each discrepancy appears in separate columns.

    Args:
        differences_df (DataFrame): DataFrame containing mismatched values.
        epic_df (DataFrame): The original EPIC DataFrame to determine column order.

    Returns:
        DataFrame: A structured DataFrame where mismatches are arranged in a single row per patient.
    """
    # Standardize column names to prevent mismatches
    differences_df.rename(columns=lambda x: x.strip(), inplace=True)

    required_columns = ["FID", "SSR", "EPIC Column", "SecuTrial Column", "EPIC Value", "SecuTrial Value"]
    missing_columns = [col for col in required_columns if col not in differences_df.columns]

    if missing_columns:
        raise ValueError(f"Missing required columns in differences_df: {missing_columns}")

    # Resolve duplicates by taking the first occurrence
    duplicate_check = differences_df.duplicated(subset=["FID", "SSR", "EPIC Column"], keep=False)
    if duplicate_check.any():
        print(f"Warning: {duplicate_check.sum()} duplicate rows found. Resolving by taking the first occurrence.")

    differences_df = differences_df.groupby(["FID", "SSR", "EPIC Column"], as_index=False).first()

    # Pivot the table to make each discrepancy a separate column
    pivoted_df = differences_df.pivot(index=["FID", "SSR"], 
                                      columns="EPIC Column", 
                                      values=["SecuTrial Value", "EPIC Value"])

    # Flatten multi-level column names
    pivoted_df.columns = [f"{col[1]}_st" if col[0] == "SecuTrial Value" else f"{col[1]}_ep" 
                          for col in pivoted_df.columns]

    # Reset index to include FID and SSR as columns
    pivoted_df.reset_index(inplace=True)

    # Debug: Print column names after pivot
    #print("Columns after pivot:", pivoted_df.columns)

    # Ensure column order follows the order in the original EPIC DataFrame
    column_order = ["FID", "SSR"]

    # Extract base column names from epic_df (without prefix/suffix)
    base_columns = [col for col in epic_df.columns if not col.startswith(("FID", "SSR"))]

    # Ensure `_st` (SecuTrial) columns appear first, then `_ep` (EPIC) columns
    for col in base_columns:
        if f"{col}_st" in pivoted_df.columns:
            column_order.append(f"{col}_st")
        if f"{col}_ep" in pivoted_df.columns:
            column_order.append(f"{col}_ep")

    # Debug: Check if any expected columns are missing
    missing_expected_columns = [col for col in column_order if col not in pivoted_df.columns]
    if missing_expected_columns:
        print(f"⚠️ Warning: Some expected columns are missing after pivot: {missing_expected_columns}")

    # Select only available columns and reorder
    column_order = [col for col in column_order if col in pivoted_df.columns]
    pivoted_df = pivoted_df[column_order]

    # Fill NaN values with an empty string for better readability
    pivoted_df.fillna("", inplace=True)

    return pivoted_df

In [48]:
restructured_df = restructure_mismatched_data(mismatched_results, df_EPIC_all_2)

In [49]:
restructured_df.head()

,FID,SSR,enct.name_last_st,enct.name_last_ep,enct.name_first_st,enct.name_first_ep,enct.birth_date_st,enct.birth_date_ep,enct.zip_st,enct.zip_ep,...,med.treat_antiplatelet_st,med.treat_antiplatelet_ep,med.treat_anticoagulant_st,med.treat_anticoagulant_ep,med.treat_ivt_st,med.treat_ivt_ep,mon.GROIN_PUNCTURE_st,mon.GROIN_PUNCTURE_ep,mon.DOOR_TO_GROIN_st,mon.DOOR_TO_GROIN_ep
0,10105076,13681,,,,,19580401 00:00,19580104 00:00,,,...,,,,,,,,,,
1,10105089,13672,,,Katja,Katia,,,,,...,,,no,yes,started before admission,no,20250414 09:13,20250414 15:00,68,71
2,10105149,13682,,,,,19440721 00:00,19460721 00:00,,,...,,,,,,,,,,
3,10105259,13673,,,,,19300504 00:00,19300405 00:00,,,...,no,yes,no,yes,started before admission,no,20250414 16:36,20250414 00:00,,
4,10105411,13674,,,,,,,,,...,no,yes,no,yes,no,yes,,,,


## Save new EPIC and sT dataframe as Excel


In [50]:
# Define output directory and ensure it exists
output_dir = base_dir / 'EPIC-export-validation/validation-files'
output_dir.mkdir(parents=True, exist_ok=True)

# Generate timestamped file name and path
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_file_path = output_dir / f"df_EPIC_all_{timestamp}.xlsx"

# Save the DataFrame
try:
    df_EPIC_all.to_excel(output_file_path, index=False)
    print(f"File saved successfully at: {output_file_path}")
except Exception as e:
    print(f"Error saving file: {e}")

File saved successfully at: /Users/yaskhanloo/Developer/bern-storke-center/EPIC-export-validation/validation-files/df_EPIC_all_20250414_095129.xlsx


In [51]:
# Generate the output file path
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_file_path = output_dir / f"df_secuTrial_w_REVAS_{timestamp}.xlsx"

# Save the DataFrame
try:
    df_secuTrial_w_REVAS.to_excel(output_file_path, index=False)
    print(f"File saved successfully at: {output_file_path}")
except Exception as e:
    print(f"Error saving file: {e}")

File saved successfully at: /Users/yaskhanloo/Developer/bern-storke-center/EPIC-export-validation/validation-files/df_secuTrial_w_REVAS_20250414_095131.xlsx


In [52]:
# Generate the output file path
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_file_path = output_dir / f"report_mismatched_values_{timestamp}.xlsx"

# Save the DataFrame
try:
    restructured_df.to_excel(output_file_path, index=False)
    print(f"File saved successfully at: {output_file_path}")
except Exception as e:
    print(f"Error saving file: {e}")

File saved successfully at: /Users/yaskhanloo/Developer/bern-storke-center/EPIC-export-validation/validation-files/report_mismatched_values_20250414_095135.xlsx
